In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import random

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# 1. 데이터 로드 (예시)
seoul_data = pd.read_csv('../서울대기질.csv', parse_dates=['date'], index_col='date')
liaocheng_data = pd.read_csv('../랴오청대기질.csv', parse_dates=['date'], index_col='date')

# 컬럼 이름의 공백 제거
seoul_data.columns = seoul_data.columns.str.replace(' ', '')
liaocheng_data.columns = liaocheng_data.columns.str.replace(' ', '')

# 데이터 샘플 확인
print(seoul_data.head())
print(liaocheng_data.head())

# 공백을 NaN으로 변환
seoul_data.replace(" ", np.nan, inplace=True)
liaocheng_data.replace(" ", np.nan, inplace=True)

# 결측치 평균으로 대체
seoul_data = seoul_data.apply(pd.to_numeric)  # 데이터 타입이 숫자형인지 확인 및 변환
liaocheng_data = liaocheng_data.apply(pd.to_numeric)
seoul_data.fillna(seoul_data.mean(), inplace=True)
liaocheng_data.fillna(liaocheng_data.mean(), inplace=True)

# 사용하려는 컬럼 이름 (여기서는 'pm25'와 'pm10'을 사용합니다.)
columns_to_predict = ['pm25', 'pm10']

for column in columns_to_predict:
    # 2. 데이터 시각화
    plt.figure(figsize=(14, 7))
    plt.plot(seoul_data.index, seoul_data[column], label=f'Seoul {column.upper()}')
    plt.plot(liaocheng_data.index, liaocheng_data[column], label=f'Liaocheng {column.upper()}')
    plt.legend()
    plt.title(f'Seoul vs Liaocheng {column.upper()}')
    plt.show()

    # 3. 상관관계 분석
    merged_data = pd.merge(seoul_data[[column]], liaocheng_data[[column]], left_index=True, right_index=True, suffixes=('_seoul', '_liaocheng'))
    correlation = merged_data.corr()
    print(f'Correlation for {column.upper()}')
    print(correlation)

    # 상관 행렬 시각화
    sns.heatmap(correlation, annot=True, cmap='coolwarm')
    plt.title(f'Correlation Matrix for {column.upper()}')
    plt.show()

    # 4. 시계열 모델링 및 예측
    # SARIMA 모델링
    # 서울 예측
    seoul_model = SARIMAX(seoul_data[column], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    seoul_results = seoul_model.fit()

    # 랴오청 예측
    liaocheng_model = SARIMAX(liaocheng_data[column], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    liaocheng_results = liaocheng_model.fit()

    # 미래 예측 (예: 30일)
    seoul_forecast = seoul_results.get_forecast(steps=30)
    liaocheng_forecast = liaocheng_results.get_forecast(steps=30)

    # 예측값 시각화
    plt.figure(figsize=(14, 7))
    plt.plot(seoul_data.index, seoul_data[column], label=f'Seoul {column.upper()}')
    plt.plot(seoul_forecast.predicted_mean.index, seoul_forecast.predicted_mean, label=f'Seoul Forecast', linestyle='--')
    plt.plot(liaocheng_data.index, liaocheng_data[column], label=f'Liaocheng {column.upper()}')
    plt.plot(liaocheng_forecast.predicted_mean.index, liaocheng_forecast.predicted_mean, label=f'Liaocheng Forecast', linestyle='--')
    plt.legend()
    plt.title(f'{column.upper()} Forecast for Seoul and Liaocheng')
    plt.show()

    # 예측 오차 계산 (예: 과거 데이터로 검증)
    train_seoul = seoul_data[:-30]
    test_seoul = seoul_data[-30:]
    seoul_model = SARIMAX(train_seoul[column], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    seoul_results = seoul_model.fit()
    seoul_pred = seoul_results.get_forecast(steps=30)
    mse_seoul = mean_squared_error(test_seoul[column], seoul_pred.predicted_mean)
    print(f'Seoul {column.upper()} Forecast MSE: {mse_seoul}')

    train_liaocheng = liaocheng_data[:-30]
    test_liaocheng = liaocheng_data[-30:]
    liaocheng_model = SARIMAX(train_liaocheng[column], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    liaocheng_results = liaocheng_model.fit()
    liaocheng_pred = liaocheng_results.get_forecast(steps=30)
    mse_liaocheng = mean_squared_error(test_liaocheng[column], liaocheng_pred.predicted_mean)
    print(f'Liaocheng {column.upper()} Forecast MSE: {mse_liaocheng}')

FileNotFoundError: [Errno 2] No such file or directory: '../서울대기질.csv'

In [6]:
seoul_data.columns

Index(['pm25', 'pm10', 'o3', 'no2', 'so2', 'co'], dtype='object')

In [ ]:


# 데이터 불러오기
beijing = pd.read_csv('../랴오청대기질.csv')
seoul = pd.read_csv('../서울대기질.csv')

# 데이터 구조 확인
print("Beijing data structure:")
print(beijing.head())
print("\nSeoul data structure:")
print(seoul.head())

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체
beijing_numeric = beijing.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
seoul_numeric = seoul.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')

# ' '을 NaN으로 변환
beijing_numeric.replace(' ', np.nan, inplace=True)
seoul_numeric.replace(' ', np.nan, inplace=True)

# NaN 값을 평균값으로 대체
beijing_numeric.fillna(beijing_numeric.mean(), inplace=True)
seoul_numeric.fillna(seoul_numeric.mean(), inplace=True)

# 각 도시의 PM2.5와 PM10의 상관관계 분석
plt.figure(figsize=(12, 8))
sns.heatmap(beijing_numeric.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix for Beijing')
plt.show()

plt.figure(figsize=(12, 8))
sns.heatmap(seoul_numeric.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix for Seoul')
plt.show()

# 상관관계 파악을 위해 병합
corr_beijing_seoul = pd.merge(beijing, seoul, on='date', suffixes=('_beijing', '_seoul'))

# 병합된 데이터프레임의 열 이름 확인
print("\nColumns in corr_beijing_seoul:")
print(corr_beijing_seoul.columns)

# 공백을 포함한 컬럼 이름들을 모두 수정
corr_beijing_seoul.columns = corr_beijing_seoul.columns.str.replace(' ', '')

# 수정된 컬럼 이름 확인
print(corr_beijing_seoul.columns)

# 예측 모델 생성 (서울의 PM2.5와 PM10을 예측)
# PM2.5 예측 모델
X_pm25 = corr_beijing_seoul[['pm25_beijing', 'pm10_beijing']]  # 실제 데이터프레임의 컬럼 이름으로 수정
y_pm25 = corr_beijing_seoul['pm25_seoul']  # 서울의 PM2.5

# 공백을 NaN으로 변환
X_pm25.replace(' ', np.nan, inplace=True)
y_pm25.replace(' ', np.nan, inplace=True)

# NaN 값을 평균값으로 대체
X_pm25.fillna(X_pm25.astype(float).mean(), inplace=True)
y_pm25.fillna(y_pm25.astype(float).mean(), inplace=True)

X_train_pm25, X_test_pm25, y_train_pm25, y_test_pm25 = train_test_split(X_pm25, y_pm25, test_size=0.2, random_state=42)

model_pm25 = LinearRegression()
model_pm25.fit(X_train_pm25, y_train_pm25)

y_pred_pm25 = model_pm25.predict(X_test_pm25)

print(f"\nPM2.5 Prediction Model Performance:")
print(f"Mean Squared Error: {mean_squared_error(y_test_pm25, y_pred_pm25)}")
print(f"R^2 Score: {r2_score(y_test_pm25, y_pred_pm25)}")

# PM10 예측 모델
X_pm10 = corr_beijing_seoul[['pm25_beijing', 'pm10_beijing']]
y_pm10 = corr_beijing_seoul['pm10_seoul']

# 공백을 NaN으로 변환
X_pm10.replace(' ', np.nan, inplace=True)
y_pm10.replace(' ', np.nan, inplace=True)

# NaN 값을 평균값으로 대체
X_pm10.fillna(X_pm10.astype(float).mean(), inplace=True)
y_pm10.fillna(y_pm10.astype(float).mean(), inplace=True)

X_train_pm10, X_test_pm10, y_train_pm10, y_test_pm10 = train_test_split(X_pm10, y_pm10, test_size=0.2, random_state=42)

model_pm10 = LinearRegression()
model_pm10.fit(X_train_pm10, y_train_pm10)

y_pred_pm10 = model_pm10.predict(X_test_pm10)

print(f"\nPM10 Prediction Model Performance:")
print(f"Mean Squared Error: {mean_squared_error(y_test_pm10, y_pred_pm10)}")
print(f"R^2 Score: {r2_score(y_test_pm10, y_pred_pm10)}")

# Convert y_test and y_pred_pm25 to float if they are not already
y_test_pm25 = y_test_pm25.astype(float)
y_pred_pm25 = y_pred_pm25.astype(float)

# Plotting
plt.figure(figsize=(14, 6))
plt.plot(y_test_pm25.values, label='Actual PM2.5')
plt.plot(y_pred_pm25, label='Predicted PM2.5')
plt.legend()
plt.xlabel('Index')
plt.ylabel('PM2.5')
plt.title('Actual vs Predicted PM2.5')
plt.grid(True)
plt.show()

plt.figure(figsize=(14, 6))
plt.plot(y_test_pm10.values, label='Actual PM10')
plt.plot(y_pred_pm10, label='Predicted PM10')
plt.legend()
plt.title('Actual vs Predicted PM10')
plt.show()

# 서울의 PM2.5와 PM10 예측 (2024.06.22 ~ 2024.06.30)
future_dates = pd.date_range(start='2024-06-22', end='2024-06-30').to_frame(index=False, name='date')

# 베이징의 미래 데이터를 임의로 생성 (실제 데이터를 사용할 수 있다면 그것을 사용)
future_beijing = pd.DataFrame({
    'date': future_dates['date'],
    'pm25_beijing': [random.randint(50, 150) for _ in range(len(future_dates))],
    'pm10_beijing': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# 서울의 PM2.5와 PM10 예측
future_X_pm25 = future_beijing[['pm25_beijing', 'pm10_beijing']]
future_pred_pm25 = model_pm25.predict(future_X_pm25)

future_X_pm10 = future_beijing[['pm25_beijing', 'pm10_beijing']]
future_pred_pm10 = model_pm10.predict(future_X_pm10)

# 결과 출력
future_beijing['Predicted_PM25'] = future_pred_pm25
future_beijing['Predicted_PM10'] = future_pred_pm10

print("\nFuture PM2.5 and PM10 predictions for Seoul (2024.06.22 ~ 2024.06.30):")
print(future_beijing[['date', 'Predicted_PM25', 'Predicted_PM10']])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\FILE\랴오청대기질.csv')
seoul = pd.read_csv('D:\FILE\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing)
seoul_numeric = preprocess(seoul)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

# PM2.5 예측 모델
X_pm25 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm25 = corr_beijing_seoul[' pm25_seoul']

# PM10 예측 모델
X_pm10 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm10 = corr_beijing_seoul[' pm10_seoul']

# 데이터 준비 함수
def prepare_data(X, y):
    X = X.copy()
    y = y.copy()
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.astype(float).mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.astype(float).mean(), inplace=True)
    return X.astype(float), y.astype(float)

# PM2.5, PM10 데이터 준비
X_pm25, y_pm25 = prepare_data(X_pm25, y_pm25)
X_pm10, y_pm10 = prepare_data(X_pm10, y_pm10)

# 모델 생성 및 학습
model_pm25 = LinearRegression()
model_pm25.fit(X_pm25, y_pm25)

model_pm10 = LinearRegression()
model_pm10.fit(X_pm10, y_pm10)

# 2024년 6월 1일부터 30일까지의 데이터 생성
future_dates = pd.date_range(start='2024-06-01', end='2024-06-30')
future_beijing = pd.DataFrame({
    'date': future_dates,
    ' pm25_beijing': [random.randint(50, 150) for _ in range(len(future_dates))],
    ' pm10_beijing': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# PM2.5, PM10 예측
future_X_pm25 = future_beijing[[' pm25_beijing', ' pm10_beijing']]
future_X_pm10 = future_beijing[[' pm25_beijing', ' pm10_beijing']]

future_pred_pm25 = model_pm25.predict(future_X_pm25)
future_pred_pm10 = model_pm10.predict(future_X_pm10)

future_beijing['Predicted_PM25'] = future_pred_pm25
future_beijing['Predicted_PM10'] = future_pred_pm10

# AQI를 μg/m³로 변환하는 함수 정의 (한국 기준)
def aqi_conversion(pm_value, pm_type):
    if pm_type == 'PM2.5':
        if pm_value <= 12:
            return pm_value * (15.0 / 12)
        elif pm_value <= 35.4:
            return 15.0 + (pm_value - 12) * ((35.0 - 15.0) / (35.4 - 12))
        elif pm_value <= 75.4:
            return 35.0 + (pm_value - 35.4) * ((75.0 - 35.0) / (75.4 - 35.4))
        elif pm_value <= 115.4:
            return 75.0 + (pm_value - 75.4) * ((115.0 - 75.0) / (115.4 - 75.4))
        elif pm_value <= 150.4:
            return 115.0 + (pm_value - 115.4) * ((150.0 - 115.0) / (150.4 - 115.4))
        elif pm_value <= 500:
            return 150.0 + (pm_value - 150.4) * ((350.0 - 150.0) / (500 - 150.4))
        else:
            return 500.0
    elif pm_type == 'PM10':
        if pm_value <= 30:
            return pm_value * (30.0 / 30)
        elif pm_value <= 80:
            return 30.0 + (pm_value - 30) * ((80.0 - 30.0) / (80 - 30))
        elif pm_value <= 150:
            return 80.0 + (pm_value - 80) * ((150.0 - 80.0) / (150 - 80))
        elif pm_value <= 250:
            return 150.0 + (pm_value - 150) * ((250.0 - 150.0) / (250 - 150))
        elif pm_value <= 350:
            return 250.0 + (pm_value - 250) * ((350.0 - 250.0) / (350 - 250))
        elif pm_value <= 420:
            return 350.0 + (pm_value - 350) * ((420.0 - 350.0) / (420 - 350))
        else:
            return 420.0

# # 예측 데이터를 AQI로 변환
future_beijing['Predicted_AQI_PM25'] = future_beijing['Predicted_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
future_beijing['Predicted_AQI_PM10'] = future_beijing['Predicted_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))

# # 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-06-01') & (seoul['date'] <= '2024-06-30')][['date', ' pm25', ' pm10']].reset_index(drop=True)
actual_seoul.columns = ['date', 'Actual_PM25', 'Actual_PM10']

# 'Actual_PM25'와 'Actual_PM10' 열을 숫자로 변환
actual_seoul['Actual_PM25'] = pd.to_numeric(actual_seoul['Actual_PM25'], errors='coerce')
actual_seoul['Actual_PM10'] = pd.to_numeric(actual_seoul['Actual_PM10'], errors='coerce')

# # 데이터를 AQI로 변환
actual_seoul['Actual_AQI_PM25'] = actual_seoul['Actual_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
actual_seoul['Actual_AQI_PM10'] = actual_seoul['Actual_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))

# # 오차 비율 계산
combined_data = pd.merge(future_beijing, actual_seoul, on='date', how='left')
combined_data['PM25_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM25'] - combined_data['Predicted_AQI_PM25']) / combined_data['Actual_AQI_PM25']) * 100
combined_data['PM10_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM10'] - combined_data['Predicted_AQI_PM10']) / combined_data['Actual_AQI_PM10']) * 100

# 함수 정의: AQI 값을 입력받아 범주를 반환하는 함수
def categorize_aqi(aqi_value):
    if aqi_value <= 50:
        return 1
    elif aqi_value <= 100:
        return 2
    elif aqi_value <= 150:
        return 3
    else:
        return 4

# 예측된 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_predicted_aqi(df):
    df['Predicted_AQI_PM25_Category'] = df['Predicted_AQI_PM25'].apply(categorize_aqi)
    df['Predicted_AQI_PM10_Category'] = df['Predicted_AQI_PM10'].apply(categorize_aqi)
    return df

# 실제 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_actual_aqi(df):
    df['Actual_AQI_PM25_Category'] = df['Actual_AQI_PM25'].apply(categorize_aqi)
    df['Actual_AQI_PM10_Category'] = df['Actual_AQI_PM10'].apply(categorize_aqi)
    return df

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories(row):
    if row['Predicted_AQI_PM25_Category'] != row['Actual_AQI_PM25_Category']:
        row['PM25_Category_Difference'] = 1
    else:
        row['PM25_Category_Difference'] = 0
    
    if row['Predicted_AQI_PM10_Category'] != row['Actual_AQI_PM10_Category']:
        row['PM10_Category_Difference'] = 1
    else:
        row['PM10_Category_Difference'] = 0
    
    return row

# COMBINED_DATA에 적용
combined_data_categorized = categorize_predicted_aqi(combined_data)
combined_data_categorized = categorize_actual_aqi(combined_data_categorized)

# combined_data_categorized에 함수 적용
combined_data_categorized_diff = combined_data_categorized.apply(compare_aqi_categories, axis=1)

# 결과 출력
print("\n범주화된 예측된 AQI 데이터 및 실제 AQI 데이터:")
# combined_data_categorized_diff[['date', 'Predicted_AQI_PM25_Category', 'Actual_AQI_PM25_Category', 
#                                       'PM25_Category_Difference', 'Predicted_AQI_PM10_Category', 
#                                       'Actual_AQI_PM10_Category', 'PM10_Category_Difference']]

# 6월 한 달 동안의 데이터 필터링
june_data = combined_data_categorized_diff[(combined_data_categorized_diff['date'] >= '2024-06-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-06-30')]

# PM2.5 및 PM10의 카테고리가 같은 비율 계산
total_days = len(june_data)
same_pm25_category_count = june_data[june_data['PM25_Category_Difference'] == 0].shape[0]
same_pm10_category_count = june_data[june_data['PM10_Category_Difference'] == 0].shape[0]

percent_same_pm25_category = (same_pm25_category_count / total_days) * 100
percent_same_pm10_category = (same_pm10_category_count / total_days) * 100

# 결과 출력
print(f"\n6월 한 달 동안 PM2.5의 예측 카테고리가 실제와 같은 비율: {percent_same_pm25_category:.2f}%")
print(f"6월 한 달 동안 PM10의 예측 카테고리가 실제와 같은 비율: {percent_same_pm10_category:.2f}%")

june_data
# # 시각화 예시 (선택사항)
# # 예측된 AQI 및 실제 AQI 범주 분포를 시각화하여 확인할 수 있습니다.
# plt.figure(figsize=(16, 8))

# plt.subplot(2, 2, 1)
# sns.histplot(combined_data_categorized['Predicted_AQI_PM25_Category'], bins=range(1, 6), kde=True, color='blue', label='Predicted PM2.5')
# plt.title('Predicted PM2.5 AQI Category Distribution')
# plt.xlabel('AQI Category')
# plt.ylabel('Frequency')
# plt.xticks([1, 2, 3, 4], ['1-50', '51-100', '101-150', '151+'])
# plt.legend()

# plt.subplot(2, 2, 2)
# sns.histplot(combined_data_categorized['Actual_AQI_PM25_Category'], bins=range(1, 6), kde=True, color='green', label='Actual PM2.5')
# plt.title('Actual PM2.5 AQI Category Distribution')
# plt.xlabel('AQI Category')
# plt.ylabel('Frequency')
# plt.xticks([1, 2, 3, 4], ['1-50', '51-100', '101-150', '151+'])
# plt.legend()

# plt.subplot(2, 2, 3)
# sns.histplot(combined_data_categorized['Predicted_AQI_PM10_Category'], bins=range(1, 6), kde=True, color='red', label='Predicted PM10')
# plt.title('Predicted PM10 AQI Category Distribution')
# plt.xlabel('AQI Category')
# plt.ylabel('Frequency')
# plt.xticks([1, 2, 3, 4], ['1-50', '51-100', '101-150', '151+'])
# plt.legend()

# plt.subplot(2, 2, 4)
# sns.histplot(combined_data_categorized['Actual_AQI_PM10_Category'], bins=range(1, 6), kde=True, color='orange', label='Actual PM10')
# plt.title('Actual PM10 AQI Category Distribution')
# plt.xlabel('AQI Category')
# plt.ylabel('Frequency')
# plt.xticks([1, 2, 3, 4], ['1-50', '51-100', '101-150', '151+'])
# plt.legend()

# plt.tight_layout()
# plt.show()

In [ ]:
cm

In [84]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2024년 5월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

# PM2.5 예측 모델
X_pm25 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm25 = corr_beijing_seoul[' pm25_seoul']

# PM10 예측 모델
X_pm10 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm10 = corr_beijing_seoul[' pm10_seoul']

# 데이터 준비 함수
def prepare_data(X, y):
    X = X.copy()
    y = y.copy()
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.astype(float).mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.astype(float).mean(), inplace=True)
    return X.astype(float), y.astype(float)

# PM2.5, PM10 데이터 준비
X_pm25, y_pm25 = prepare_data(X_pm25, y_pm25)
X_pm10, y_pm10 = prepare_data(X_pm10, y_pm10)

# 모델 생성 및 학습
model_pm25 = LinearRegression()
model_pm25.fit(X_pm25, y_pm25)

model_pm10 = LinearRegression()
model_pm10.fit(X_pm10, y_pm10)

# 2024년 6월 1일부터 30일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-06-30')
future_beijing = pd.DataFrame({
    'date': future_dates,
    ' pm25_beijing': [random.randint(50, 150) for _ in range(len(future_dates))],
    ' pm10_beijing': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# PM2.5, PM10 예측
future_X_pm25 = future_beijing[[' pm25_beijing', ' pm10_beijing']]
future_X_pm10 = future_beijing[[' pm25_beijing', ' pm10_beijing']]

future_pred_pm25 = model_pm25.predict(future_X_pm25)
future_pred_pm10 = model_pm10.predict(future_X_pm10)

future_beijing['Predicted_PM25'] = future_pred_pm25
future_beijing['Predicted_PM10'] = future_pred_pm10

# AQI를 μg/m³로 변환하는 함수 정의 (한국 기준)
def aqi_conversion(pm_value, pm_type):
    if pm_type == 'PM2.5':
        if pm_value <= 12:
            return pm_value * (15.0 / 12)
        elif pm_value <= 35.4:
            return 15.0 + (pm_value - 12) * ((35.0 - 15.0) / (35.4 - 12))
        elif pm_value <= 75.4:
            return 35.0 + (pm_value - 35.4) * ((75.0 - 35.0) / (75.4 - 35.4))
        elif pm_value <= 115.4:
            return 75.0 + (pm_value - 75.4) * ((115.0 - 75.0) / (115.4 - 75.4))
        elif pm_value <= 150.4:
            return 115.0 + (pm_value - 115.4) * ((150.0 - 115.0) / (150.4 - 115.4))
        elif pm_value <= 500:
            return 150.0 + (pm_value - 150.4) * ((350.0 - 150.0) / (500 - 150.4))
        else:
            return 500.0
    elif pm_type == 'PM10':
        if pm_value <= 30:
            return pm_value * (30.0 / 30)
        elif pm_value <= 80:
            return 30.0 + (pm_value - 30) * ((80.0 - 30.0) / (80 - 30))
        elif pm_value <= 150:
            return 80.0 + (pm_value - 80) * ((150.0 - 80.0) / (150 - 80))
        elif pm_value <= 250:
            return 150.0 + (pm_value - 150) * ((250.0 - 150.0) / (250 - 150))
        elif pm_value <= 350:
            return 250.0 + (pm_value - 250) * ((350.0 - 250.0) / (350 - 250))
        elif pm_value <= 420:
            return 350.0 + (pm_value - 350) * ((420.0 - 350.0) / (420 - 350))
        else:
            return 420.0

# 예측 데이터를 AQI로 변환
future_beijing['Predicted_AQI_PM25'] = future_beijing['Predicted_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
future_beijing['Predicted_AQI_PM10'] = future_beijing['Predicted_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-06-30')][['date', ' pm25', ' pm10']].reset_index(drop=True)
actual_seoul.columns = ['date', 'Actual_PM25', 'Actual_PM10']

# 'Actual_PM25'와 'Actual_PM10' 열을 숫자로 변환
actual_seoul['Actual_PM25'] = pd.to_numeric(actual_seoul['Actual_PM25'], errors='coerce')
actual_seoul['Actual_PM10'] = pd.to_numeric(actual_seoul['Actual_PM10'], errors='coerce')

# 데이터를 AQI로 변환
actual_seoul['Actual_AQI_PM25'] = actual_seoul['Actual_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
actual_seoul['Actual_AQI_PM10'] = actual_seoul['Actual_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))

# 오차 비율 계산
combined_data = pd.merge(future_beijing, actual_seoul, on='date', how='left')
combined_data['PM25_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM25'] - combined_data['Predicted_AQI_PM25']) / combined_data['Actual_AQI_PM25']) * 100
combined_data['PM10_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM10'] - combined_data['Predicted_AQI_PM10']) / combined_data['Actual_AQI_PM10']) * 100

# 함수 정의: AQI 값을 입력받아 범주를 반환하는 함수
def categorize_aqi(aqi_value):
    if aqi_value <= 50:
        return 1
    elif aqi_value <= 100:
        return 2
    elif aqi_value <= 150:
        return 3
    else:
        return 4

# 예측된 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_predicted_aqi(df):
    df['Predicted_AQI_PM25_Category'] = df['Predicted_AQI_PM25'].apply(categorize_aqi)
    df['Predicted_AQI_PM10_Category'] = df['Predicted_AQI_PM10'].apply(categorize_aqi)
    return df

# 실제 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_actual_aqi(df):
    df['Actual_AQI_PM25_Category'] = df['Actual_AQI_PM25'].apply(categorize_aqi)
    df['Actual_AQI_PM10_Category'] = df['Actual_AQI_PM10'].apply(categorize_aqi)
    return df

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories(row):
    if row['Predicted_AQI_PM25_Category'] != row['Actual_AQI_PM25_Category']:
        row['PM25_Category_Difference'] = 1
    else:
        row['PM25_Category_Difference'] = 0
    
    if row['Predicted_AQI_PM10_Category'] != row['Actual_AQI_PM10_Category']:
        row['PM10_Category_Difference'] = 1
    else:
        row['PM10_Category_Difference'] = 0
    
    return row

# COMBINED_DATA에 적용
combined_data_categorized = categorize_predicted_aqi(combined_data)
combined_data_categorized = categorize_actual_aqi(combined_data_categorized)

# combined_data_categorized에 함수 적용
combined_data_categorized_diff = combined_data_categorized.apply(compare_aqi_categories, axis=1)

# 결과 출력
print("\n범주화된 예측된 AQI 데이터 및 실제 AQI 데이터:")
# combined_data_categorized_diff[['date', 'Predicted_AQI_PM25_Category', 'Actual_AQI_PM25_Category', 
#                                       'PM25_Category_Difference', 'Predicted_AQI_PM10_Category', 
#                                       'Actual_AQI_PM10_Category', 'PM10_Category_Difference']]

# 6월 한 달 동안의 데이터 필터링
june_data = combined_data_categorized_diff[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-06-30')]

# PM2.5 및 PM10의 카테고리가 같은 비율 계산
total_days = len(june_data)
same_pm25_category_count = june_data[june_data['PM25_Category_Difference'] == 0].shape[0]
same_pm10_category_count = june_data[june_data['PM10_Category_Difference'] == 0].shape[0]

percent_same_pm25_category = (same_pm25_category_count / total_days) * 100
percent_same_pm10_category = (same_pm10_category_count / total_days) * 100

# 결과 출력
print(f"\n2024년 동안 PM2.5의 예측 카테고리가 실제와 같은 비율: {percent_same_pm25_category:.2f}%")
print(f"2024년 동안 PM 10의 예측 카테고리가 실제와 같은 비율: {percent_same_pm10_category:.2f}%")
june_data


범주화된 예측된 AQI 데이터 및 실제 AQI 데이터:

2024년 동안 PM2.5의 예측 카테고리가 실제와 같은 비율: 60.44%
2024년 동안 PM 10의 예측 카테고리가 실제와 같은 비율: 72.53%


,date,pm25_beijing,pm10_beijing,Predicted_PM25,Predicted_PM10,Predicted_AQI_PM25,Predicted_AQI_PM10,Actual_PM25,Actual_PM10,Actual_AQI_PM25,Actual_AQI_PM10,PM25_Error_Rate,PM10_Error_Rate,Predicted_AQI_PM25_Category,Predicted_AQI_PM10_Category,Actual_AQI_PM25_Category,Actual_AQI_PM10_Category,PM25_Category_Difference,PM10_Category_Difference
0,2024-01-01,80,66,78.601958,43.419869,78.201958,43.419869,81,41,80.6,41.0,2.975238,5.902120,2,1,2,1,0,0
1,2024-01-02,131,52,86.487191,43.999649,86.087191,43.999649,103,44,102.6,44.0,16.094356,0.000798,2,1,3,1,1,0
2,2024-01-03,108,30,80.275910,38.832774,79.875910,38.832774,102,55,101.6,55.0,21.381978,29.394957,2,1,3,2,1,1
3,2024-01-04,150,47,89.445053,44.253013,89.045053,44.253013,124,71,123.6,71.0,27.957077,37.671812,2,1,3,2,1,1
4,2024-01-05,87,79,81.083555,46.084636,80.683555,46.084636,149,28,148.6,28.0,45.704203,64.587987,2,1,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,2024-06-26,113,55,83.522129,43.458708,83.122129,43.458708,61,34,60.6,34.0,37.165229,27.819730,2,1,2,1,0,0
178,2024-06-27,125,47,84.936171,42.779824,84.536171,42.779824,66,46,65.6,46.0,28.866114,7.000382,2,1,2,1,0,0
179,2024-06-28,124,60,85.974925,44.973171,85.574925,44.973171,94,33,93.6,33.0,8.573798,36.282336,2,1,2,1,0,0
180,2024-06-29,105,58,82.360619,43.507043,81.960619,43.507043,79,20,78.6,20.0,4.275597,117.535217,2,1,2,1,0,0


In [46]:
# 특정 날짜의 예측 값을 출력하는 함수 정의
def get_predictions_for_date(date, combined_data):
    # 날짜 형식을 datetime으로 변환
    date = pd.to_datetime(date, errors='coerce')
    
    # 해당 날짜의 데이터 필터링
    data_for_date = combined_data[combined_data['date'] == date]
    
    if data_for_date.empty:
        print(f"날짜 {date.strftime('%Y-%m-%d')} 에 해당하는 데이터가 없습니다.")
        return None
    else:
        return data_for_date[['date', 'Predicted_AQI_PM25', 'Actual_AQI_PM25', 'PM25_Error_Rate', 
                              'Predicted_AQI_PM10', 'Actual_AQI_PM10', 'PM10_Error_Rate', 
                              'Predicted_AQI_PM25_Category', 'Actual_AQI_PM25_Category', 'PM25_Category_Difference',
                              'Predicted_AQI_PM10_Category', 'Actual_AQI_PM10_Category', 'PM10_Category_Difference']]

# 예시 사용법
specific_date = '2024-09-30'
predictions_for_specific_date = get_predictions_for_date(specific_date, combined_data_categorized_diff)

if predictions_for_specific_date is not None:
    print(f"\n특정 날짜 ({specific_date})의 예측 및 실제 값:")
predictions_for_specific_date[['Predicted_AQI_PM25', 'Predicted_AQI_PM10', 'Predicted_AQI_PM25_Category', 'Predicted_AQI_PM10_Category']]


특정 날짜 (2024-09-30)의 예측 및 실제 값:


,Predicted_AQI_PM25,Predicted_AQI_PM10,Predicted_AQI_PM25_Category,Predicted_AQI_PM10_Category
273,93.380427,53.085328,2,2


In [98]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

# PM2.5 예측 모델
X_pm25 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm25 = corr_beijing_seoul[' pm25_seoul']

# PM10 예측 모델
X_pm10 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm10 = corr_beijing_seoul[' pm10_seoul']

# 데이터 준비 함수
def prepare_data(X, y):
    X = X.copy()
    y = y.copy()
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.astype(float).mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.astype(float).mean(), inplace=True)
    return X.astype(float), y.astype(float)

# PM2.5, PM10 데이터 준비
X_pm25, y_pm25 = prepare_data(X_pm25, y_pm25)
X_pm10, y_pm10 = prepare_data(X_pm10, y_pm10)

# 모델 생성 및 학습
model_pm25 = LinearRegression()
model_pm25.fit(X_pm25, y_pm25)

model_pm10 = LinearRegression()
model_pm10.fit(X_pm10, y_pm10)

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates,
    ' pm25_beijing': [random.randint(50, 150) for _ in range(len(future_dates))],
    ' pm10_beijing': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# PM2.5, PM10 예측
future_X_pm25 = future_beijing[[' pm25_beijing', ' pm10_beijing']]
future_X_pm10 = future_beijing[[' pm25_beijing', ' pm10_beijing']]

future_pred_pm25 = model_pm25.predict(future_X_pm25)
future_pred_pm10 = model_pm10.predict(future_X_pm10)

future_beijing['Predicted_PM25'] = future_pred_pm25
future_beijing['Predicted_PM10'] = future_pred_pm10

# AQI를 μg/m³로 변환하는 함수 정의 (한국 기준)
def aqi_conversion(pm_value, pm_type):
    if pm_type == 'PM2.5':
        if pm_value <= 12:
            return pm_value * (15.0 / 12)
        elif pm_value <= 35.4:
            return 15.0 + (pm_value - 12) * ((35.0 - 15.0) / (35.4 - 12))
        elif pm_value <= 75.4:
            return 35.0 + (pm_value - 35.4) * ((75.0 - 35.0) / (75.4 - 35.4))
        elif pm_value <= 115.4:
            return 75.0 + (pm_value - 75.4) * ((115.0 - 75.0) / (115.4 - 75.4))
        elif pm_value <= 150.4:
            return 115.0 + (pm_value - 115.4) * ((150.0 - 115.0) / (150.4 - 115.4))
        elif pm_value <= 500:
            return 150.0 + (pm_value - 150.4) * ((350.0 - 150.0) / (500 - 150.4))
        else:
            return 500.0
    elif pm_type == 'PM10':
        if pm_value <= 30:
            return pm_value * (30.0 / 30)
        elif pm_value <= 80:
            return 30.0 + (pm_value - 30) * ((80.0 - 30.0) / (80 - 30))
        elif pm_value <= 150:
            return 80.0 + (pm_value - 80) * ((150.0 - 80.0) / (150 - 80))
        elif pm_value <= 250:
            return 150.0 + (pm_value - 150) * ((250.0 - 150.0) / (250 - 150))
        elif pm_value <= 350:
            return 250.0 + (pm_value - 250) * ((350.0 - 250.0) / (350 - 250))
        elif pm_value <= 420:
            return 350.0 + (pm_value - 350) * ((420.0 - 350.0) / (420 - 350))
        else:
            return 420.0

# 예측 데이터를 AQI로 변환
future_beijing['Predicted_AQI_PM25'] = future_beijing['Predicted_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
future_beijing['Predicted_AQI_PM10'] = future_beijing['Predicted_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')][['date', ' pm25', ' pm10']].reset_index(drop=True)
actual_seoul.columns = ['date', 'Actual_PM25', 'Actual_PM10']

# 'Actual_PM25'와 'Actual_PM10' 열을 숫자로 변환
actual_seoul['Actual_PM25'] = pd.to_numeric(actual_seoul['Actual_PM25'], errors='coerce')
actual_seoul['Actual_PM10'] = pd.to_numeric(actual_seoul['Actual_PM10'], errors='coerce')

# 데이터를 AQI로 변환
actual_seoul['Actual_AQI_PM25'] = actual_seoul['Actual_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
actual_seoul['Actual_AQI_PM10'] = actual_seoul['Actual_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))

# 오차 비율 계산
combined_data = pd.merge(future_beijing, actual_seoul, on='date', how='left')
combined_data['PM25_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM25'] - combined_data['Predicted_AQI_PM25']) / combined_data['Actual_AQI_PM25']) * 100
combined_data['PM10_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM10'] - combined_data['Predicted_AQI_PM10']) / combined_data['Actual_AQI_PM10']) * 100

# 함수 정의: AQI 값을 입력받아 범주를 반환하는 함수
def categorize_aqi(aqi_value):
    if aqi_value <= 50:
        return 1
    elif aqi_value <= 100:
        return 2
    elif aqi_value <= 150:
        return 3
    else:
        return 4

# 예측된 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_predicted_aqi(df):
    df['Predicted_AQI_PM25_Category'] = df['Predicted_AQI_PM25'].apply(categorize_aqi)
    df['Predicted_AQI_PM10_Category'] = df['Predicted_AQI_PM10'].apply(categorize_aqi)
    return df

# 실제 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_actual_aqi(df):
    df['Actual_AQI_PM25_Category'] = df['Actual_AQI_PM25'].apply(categorize_aqi)
    df['Actual_AQI_PM10_Category'] = df['Actual_AQI_PM10'].apply(categorize_aqi)
    return df

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories(row):
    if row['Predicted_AQI_PM25_Category'] != row['Actual_AQI_PM25_Category']:
        row['PM25_Category_Difference'] = 1
    else:
        row['PM25_Category_Difference'] = 0
    
    if row['Predicted_AQI_PM10_Category'] != row['Actual_AQI_PM10_Category']:
        row['PM10_Category_Difference'] = 1
    else:
        row['PM10_Category_Difference'] = 0
    
    return row

# COMBINED_DATA에 적용
combined_data_categorized = categorize_predicted_aqi(combined_data)
combined_data_categorized = categorize_actual_aqi(combined_data_categorized)

# combined_data_categorized에 함수 적용
combined_data_categorized_diff = combined_data_categorized.apply(compare_aqi_categories, axis=1)

# 결과 출력
print("\n범주화된 예측된 AQI 데이터 및 실제 AQI 데이터:")
# combined_data_categorized_diff[['date', 'Predicted_AQI_PM25_Category', 'Actual_AQI_PM25_Category', 
#                                       'PM25_Category_Difference', 'Predicted_AQI_PM10_Category', 
#                                       'Actual_AQI_PM10_Category', 'PM10_Category_Difference']]

# 2024년 한 해 동안의 데이터 필터링
total_data = combined_data_categorized_diff[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-12-31')]

year_data = total_data[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-06-30')]

# PM2.5 및 PM10의 카테고리가 같은 비율 계산
total_days = len(year_data)
same_pm25_category_count = year_data[year_data['PM25_Category_Difference'] == 0].shape[0]
same_pm10_category_count = year_data[year_data['PM10_Category_Difference'] == 0].shape[0]

percent_same_pm25_category = (same_pm25_category_count / total_days) * 100
percent_same_pm10_category = (same_pm10_category_count / total_days) * 100

# 결과 출력
print(f"\n2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율: {percent_same_pm25_category:.2f}%")
print(f"2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율: {percent_same_pm10_category:.2f}%")
total_data


범주화된 예측된 AQI 데이터 및 실제 AQI 데이터:

2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율: 60.44%
2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율: 73.08%


,date,pm25_beijing,pm10_beijing,Predicted_PM25,Predicted_PM10,Predicted_AQI_PM25,Predicted_AQI_PM10,Actual_PM25,Actual_PM10,Actual_AQI_PM25,Actual_AQI_PM10,PM25_Error_Rate,PM10_Error_Rate,Predicted_AQI_PM25_Category,Predicted_AQI_PM10_Category,Actual_AQI_PM25_Category,Actual_AQI_PM10_Category,PM25_Category_Difference,PM10_Category_Difference
0,2024-01-01,98,82,83.348796,47.252595,82.948796,47.252595,81.0,41.0,80.6,41.0,2.914139,15.250232,2,1,2,1,0,0
1,2024-01-02,139,91,91.587363,51.227892,91.187363,51.227892,103.0,44.0,102.6,44.0,11.123428,16.427027,2,2,3,1,1,1
2,2024-01-03,90,48,78.717513,40.890611,78.317513,40.890611,102.0,55.0,101.6,55.0,22.915834,25.653434,2,1,3,2,1,1
3,2024-01-04,107,87,85.440882,48.649203,85.040882,48.649203,124.0,71.0,123.6,71.0,31.196697,31.479996,2,1,3,2,1,1
4,2024-01-05,73,82,78.839914,45.779406,78.439914,45.779406,149.0,28.0,148.6,28.0,47.214055,63.497879,2,1,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2024-12-27,50,75,74.035298,43.211309,73.635298,43.211309,NaN,NaN,NaN,NaN,NaN,NaN,2,1,4,4,1,1
362,2024-12-28,108,95,86.371459,50.094145,85.971459,50.094145,NaN,NaN,NaN,NaN,NaN,NaN,2,2,4,4,1,1
363,2024-12-29,67,53,75.038229,40.401537,74.638229,40.401537,NaN,NaN,NaN,NaN,NaN,NaN,2,1,4,4,1,1
364,2024-12-30,81,36,75.968983,38.281241,75.568983,38.281241,NaN,NaN,NaN,NaN,NaN,NaN,2,1,4,4,1,1


In [101]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

# PM2.5 예측 모델
X_pm25 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm25 = corr_beijing_seoul[' pm25_seoul']

# PM10 예측 모델
X_pm10 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm10 = corr_beijing_seoul[' pm10_seoul']

# 데이터 준비 함수
def prepare_data(X, y):
    X = X.copy()
    y = y.copy()
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.astype(float).mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.astype(float).mean(), inplace=True)
    return X.astype(float), y.astype(float)

# PM2.5, PM10 데이터 준비
X_pm25, y_pm25 = prepare_data(X_pm25, y_pm25)
X_pm10, y_pm10 = prepare_data(X_pm10, y_pm10)

# 모델 생성 및 학습
model_pm25 = LinearRegression()
model_pm25.fit(X_pm25, y_pm25)

model_pm10 = LinearRegression()
model_pm10.fit(X_pm10, y_pm10)

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates,
    ' pm25_beijing': [random.randint(50, 150) for _ in range(len(future_dates))],
    ' pm10_beijing': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# PM2.5, PM10 예측
future_X_pm25 = future_beijing[[' pm25_beijing', ' pm10_beijing']]
future_X_pm10 = future_beijing[[' pm25_beijing', ' pm10_beijing']]

future_pred_pm25 = model_pm25.predict(future_X_pm25)
future_pred_pm10 = model_pm10.predict(future_X_pm10)

future_beijing['Predicted_PM25'] = future_pred_pm25
future_beijing['Predicted_PM10'] = future_pred_pm10

# AQI를 μg/m³로 변환하는 함수 정의 (한국 기준)
def aqi_conversion(pm_value, pm_type):
    if pm_type == 'PM2.5':
        if pm_value <= 12:
            return pm_value * (15.0 / 12)
        elif pm_value <= 35.4:
            return 15.0 + (pm_value - 12) * ((35.0 - 15.0) / (35.4 - 12))
        elif pm_value <= 75.4:
            return 35.0 + (pm_value - 35.4) * ((75.0 - 35.0) / (75.4 - 35.4))
        elif pm_value <= 115.4:
            return 75.0 + (pm_value - 75.4) * ((115.0 - 75.0) / (115.4 - 75.4))
        elif pm_value <= 150.4:
            return 115.0 + (pm_value - 115.4) * ((150.0 - 115.0) / (150.4 - 115.4))
        elif pm_value <= 500:
            return 150.0 + (pm_value - 150.4) * ((350.0 - 150.0) / (500 - 150.4))
        else:
            return 500.0
    elif pm_type == 'PM10':
        if pm_value <= 30:
            return pm_value * (30.0 / 30)
        elif pm_value <= 80:
            return 30.0 + (pm_value - 30) * ((80.0 - 30.0) / (80 - 30))
        elif pm_value <= 150:
            return 80.0 + (pm_value - 80) * ((150.0 - 80.0) / (150 - 80))
        elif pm_value <= 250:
            return 150.0 + (pm_value - 150) * ((250.0 - 150.0) / (250 - 150))
        elif pm_value <= 350:
            return 250.0 + (pm_value - 250) * ((350.0 - 250.0) / (350 - 250))
        elif pm_value <= 420:
            return 350.0 + (pm_value - 350) * ((420.0 - 350.0) / (420 - 350))
        else:
            return 420.0

# 예측 데이터를 AQI로 변환
future_beijing['Predicted_AQI_PM25'] = future_beijing['Predicted_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
future_beijing['Predicted_AQI_PM10'] = future_beijing['Predicted_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')][['date', ' pm25', ' pm10']].reset_index(drop=True)
actual_seoul.columns = ['date', 'Actual_PM25', 'Actual_PM10']

# 'Actual_PM25'와 'Actual_PM10' 열을 숫자로 변환
actual_seoul['Actual_PM25'] = pd.to_numeric(actual_seoul['Actual_PM25'], errors='coerce')
actual_seoul['Actual_PM10'] = pd.to_numeric(actual_seoul['Actual_PM10'], errors='coerce')

# 데이터를 AQI로 변환
actual_seoul['Actual_AQI_PM25'] = actual_seoul['Actual_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
actual_seoul['Actual_AQI_PM10'] = actual_seoul['Actual_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))

# 오차 비율 계산
combined_data = pd.merge(future_beijing, actual_seoul, on='date', how='left')
combined_data['PM25_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM25'] - combined_data['Predicted_AQI_PM25']) / combined_data['Actual_AQI_PM25']) * 100
combined_data['PM10_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM10'] - combined_data['Predicted_AQI_PM10']) / combined_data['Actual_AQI_PM10']) * 100

# 함수 정의: AQI 값을 입력받아 범주를 반환하는 함수
def categorize_aqi(aqi_value):
    if aqi_value <= 50:
        return 1
    elif aqi_value <= 100:
        return 2
    elif aqi_value <= 150:
        return 3
    else:
        return 4

# 예측된 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_predicted_aqi(df):
    df['Predicted_AQI_PM25_Category'] = df['Predicted_AQI_PM25'].apply(categorize_aqi)
    df['Predicted_AQI_PM10_Category'] = df['Predicted_AQI_PM10'].apply(categorize_aqi)
    return df

# 실제 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_actual_aqi(df):
    df['Actual_AQI_PM25_Category'] = df['Actual_AQI_PM25'].apply(categorize_aqi)
    df['Actual_AQI_PM10_Category'] = df['Actual_AQI_PM10'].apply(categorize_aqi)
    return df

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories(row):
    if row['Predicted_AQI_PM25_Category'] != row['Actual_AQI_PM25_Category']:
        row['PM25_Category_Difference'] = 1
    else:
        row['PM25_Category_Difference'] = 0
    
    if row['Predicted_AQI_PM10_Category'] != row['Actual_AQI_PM10_Category']:
        row['PM10_Category_Difference'] = 1
    else:
        row['PM10_Category_Difference'] = 0
    
    return row

# COMBINED_DATA에 적용
combined_data_categorized = categorize_predicted_aqi(combined_data)
combined_data_categorized = categorize_actual_aqi(combined_data_categorized)

# combined_data_categorized에 함수 적용
combined_data_categorized_diff = combined_data_categorized.apply(compare_aqi_categories, axis=1)

# 결과 출력
print("2024년 미세먼지 예측 적중률")
# combined_data_categorized_diff[['date', 'Predicted_AQI_PM25_Category', 'Actual_AQI_PM25_Category', 
#                                       'PM25_Category_Difference', 'Predicted_AQI_PM10_Category', 
#                                       'Actual_AQI_PM10_Category', 'PM10_Category_Difference']]

# 2024년 한 해 동안의 데이터 필터링
total_data = combined_data_categorized_diff[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-12-31')]

year_data = total_data[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-06-30')]

# PM2.5 및 PM10의 카테고리가 같은 비율 계산
total_days = len(year_data)
same_pm25_category_count = year_data[year_data['PM25_Category_Difference'] == 0].shape[0]
same_pm10_category_count = year_data[year_data['PM10_Category_Difference'] == 0].shape[0]

percent_same_pm25_category = (same_pm25_category_count / total_days) * 100
percent_same_pm10_category = (same_pm10_category_count / total_days) * 100

# 결과 출력
print(f"2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율: {percent_same_pm25_category:.2f}%")
print(f"2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율: {percent_same_pm10_category:.2f}%\n")

# 특정 날짜의 데이터 출력 함수
def print_aqi_for_date(target_date):
    target_date = pd.to_datetime(target_date)
    row = combined_data[combined_data['date'] == target_date]
    if row.empty:
        print(f"No data available for the date: {target_date.strftime('%Y-%m-%d')}")
    else:
        print(f"{target_date.strftime('%Y-%m-%d')} 대기질 예측 데이터")
        print(f"예상 PM25 : {round(row['Predicted_AQI_PM25'].values[0])} (Category: {row['Predicted_AQI_PM25_Category'].values[0]})")
        print(f"예상 PM10 : {round(row['Predicted_AQI_PM10'].values[0])} (Category: {row['Predicted_AQI_PM10_Category'].values[0]})")

# 특정 날짜의 AQI 데이터를 출력
target_date = '2024-09-30'
print_aqi_for_date(target_date)

seoul

2024년 미세먼지 예측 적중률
2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율: 60.44%
2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율: 76.37%

2024-09-30 대기질 예측 데이터
예상 PM25 : 84 (Category: 2)
예상 PM10 : 46 (Category: 1)


,date,pm25,pm10,o3,no2,so2,co
0,2024-07-01,68,18,29,14,3,4
1,2024-07-02,42,29,51,10,3,5
2,2024-07-03,59,44,75,11,3,6
3,2024-07-04,108,17,51,6,3,4
4,2024-07-05,33,31,42,13,3,5
...,...,...,...,...,...,...,...
3773,2014-03-28,,93,19,49,8,6
3774,2014-03-29,,53,43,25,7,5
3775,2014-03-30,,38,22,64,8,8
3776,2014-03-31,,61,26,77,9,8


In [77]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 데이터를 필터링
beijing_past = beijing[beijing['date'] < '2024-01-01']
seoul_past = seoul[seoul['date'] < '2024-01-01']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

# 예측 모델을 위한 준비
def prepare_data(df, target):
    X = df.drop(columns=[target])
    y = df[target]
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.mean(), inplace=True)
    return X, y

# 모델 생성 및 학습
def create_and_train_model(df, target):
    X, y = prepare_data(df, target)
    model = LinearRegression()
    model.fit(X, y)
    return model

# 결측치 처리
corr_beijing_seoul.fillna(corr_beijing_seoul.mean(), inplace=True)
corr_beijing_seoul.fillna(0, inplace=True)

# 예측 모델 생성
model_pm25 = create_and_train_model(corr_beijing_seoul, ' pm25_seoul')
model_pm10 = create_and_train_model(corr_beijing_seoul, ' pm10_seoul')
model_so2 = create_and_train_model(corr_beijing_seoul, ' so2_seoul')
model_no2 = create_and_train_model(corr_beijing_seoul, ' no2_seoul')
model_co = create_and_train_model(corr_beijing_seoul, ' co_seoul')
model_o3 = create_and_train_model(corr_beijing_seoul, ' o3_seoul')

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates,
    ' pm25_beijing': [random.randint(50, 150) for _ in range(len(future_dates))],
    ' pm10_beijing': [random.randint(30, 100) for _ in range(len(future_dates))],
    ' so2_beijing': [random.randint(5, 25) for _ in range(len(future_dates))],
    ' no2_beijing': [random.randint(20, 70) for _ in range(len(future_dates))],
    ' co_beijing': [random.uniform(0.5, 2.0) for _ in range(len(future_dates))],
    ' o3_beijing': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# 예측 데이터 준비
def predict_future(model, future_data):
    return model.predict(future_data)

# 예측 수행
future_X = future_beijing.drop(columns=['date'])
future_beijing['Predicted_PM25'] = predict_future(model_pm25, future_X)
future_beijing['Predicted_PM10'] = predict_future(model_pm10, future_X)
future_beijing['Predicted_SO2'] = predict_future(model_so2, future_X)
future_beijing['Predicted_NO2'] = predict_future(model_no2, future_X)
future_beijing['Predicted_CO'] = predict_future(model_co, future_X)
future_beijing['Predicted_O3'] = predict_future(model_o3, future_X)

# AQI를 μg/m³로 변환하는 함수 정의 (한국 기준)
def aqi_conversion(pm_value, pm_type):
    if pm_type == 'PM2.5':
        if pm_value <= 12:
            return pm_value * (15.0 / 12)
        elif pm_value <= 35.4:
            return 15.0 + (pm_value - 12) * ((35.0 - 15.0) / (35.4 - 12))
        elif pm_value <= 75.4:
            return 35.0 + (pm_value - 35.4) * ((75.0 - 35.0) / (75.4 - 35.4))
        elif pm_value <= 115.4:
            return 75.0 + (pm_value - 75.4) * ((115.0 - 75.0) / (115.4 - 75.4))
        elif pm_value <= 150.4:
            return 115.0 + (pm_value - 115.4) * ((150.0 - 115.0) / (150.4 - 115.4))
        elif pm_value <= 500:
            return 150.0 + (pm_value - 150.4) * ((350.0 - 150.0) / (500 - 150.4))
        else:
            return 500.0
    elif pm_type == 'PM10':
        if pm_value <= 30:
            return pm_value * (30.0 / 30)
        elif pm_value <= 80:
            return 30.0 + (pm_value - 30) * ((80.0 - 30.0) / (80 - 30))
        elif pm_value <= 150:
            return 80.0 + (pm_value - 80) * ((150.0 - 80.0) / (150 - 80))
        elif pm_value <= 250:
            return 150.0 + (pm_value - 150) * ((250.0 - 150.0) / (250 - 150))
        elif pm_value <= 350:
            return 250.0 + (pm_value - 250) * ((350.0 - 250.0) / (350 - 250))
        elif pm_value <= 420:
            return 350.0 + (pm_value - 350) * ((420.0 - 350.0) / (420 - 350))
        else:
            return 420.0
    elif pm_type == 'SO2':
        if pm_value <= 35:
            return pm_value * (50.0 / 35)
        elif pm_value <= 75:
            return 50.0 + (pm_value - 35) * ((100.0 - 50.0) / (75 - 35))
        elif pm_value <= 185:
            return 100.0 + (pm_value - 75) * ((150.0 - 100.0) / (185 - 75))
        elif pm_value <= 304:
            return 150.0 + (pm_value - 185) * ((200.0 - 150.0) / (304 - 185))
        elif pm_value <= 604:
            return 200.0 + (pm_value - 304) * ((300.0 - 200.0) / (604 - 304))
        elif pm_value <= 1004:
            return 300.0 + (pm_value - 604) * ((400.0 - 300.0) / (1004 - 604))
        else:
            return 500.0
    elif pm_type == 'NO2':
        if pm_value <= 53:
            return pm_value * (50.0 / 53)
        elif pm_value <= 100:
            return 50.0 + (pm_value - 53) * ((100.0 - 50.0) / (100 - 53))
        elif pm_value <= 360:
            return 100.0 + (pm_value - 100) * ((150.0 - 100.0) / (360 - 100))
        elif pm_value <= 649:
            return 150.0 + (pm_value - 360) * ((200.0 - 150.0) / (649 - 360))
        elif pm_value <= 1249:
            return 200.0 + (pm_value - 649) * ((300.0 - 200.0) / (1249 - 649))
        elif pm_value <= 2049:
            return 300.0 + (pm_value - 1249) * ((400.0 - 300.0) / (2049 - 1249))
        else:
            return 500.0
    elif pm_type == 'CO':
        if pm_value <= 4.4:
            return pm_value * (50.0 / 4.4)
        elif pm_value <= 9.4:
            return 50.0 + (pm_value - 4.4) * ((100.0 - 50.0) / (9.4 - 4.4))
        elif pm_value <= 12.4:
            return 100.0 + (pm_value - 9.4) * ((150.0 - 100.0) / (12.4 - 9.4))
        elif pm_value <= 15.4:
            return 150.0 + (pm_value - 12.4) * ((200.0 - 150.0) / (15.4 - 12.4))
        elif pm_value <= 30.4:
            return 200.0 + (pm_value - 15.4) * ((300.0 - 200.0) / (30.4 - 15.4))
        elif pm_value <= 40.4:
            return 300.0 + (pm_value - 30.4) * ((400.0 - 300.0) / (40.4 - 30.4))
        else:
            return 500.0
    elif pm_type == 'O3':
        if pm_value <= 54:
            return pm_value * (50.0 / 54)
        elif pm_value <= 70:
            return 50.0 + (pm_value - 54) * ((100.0 - 50.0) / (70 - 54))
        elif pm_value <= 85:
            return 100.0 + (pm_value - 70) * ((150.0 - 100.0) / (85 - 70))
        elif pm_value <= 105:
            return 150.0 + (pm_value - 85) * ((200.0 - 150.0) / (105 - 85))
        elif pm_value <= 200:
            return 200.0 + (pm_value - 105) * ((300.0 - 200.0) / (200 - 105))
        else:
            return 500.0

# 예측 데이터를 AQI로 변환
future_beijing['Predicted_AQI_PM25'] = future_beijing['Predicted_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
future_beijing['Predicted_AQI_PM10'] = future_beijing['Predicted_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))
future_beijing['Predicted_AQI_SO2'] = future_beijing['Predicted_SO2'].apply(lambda x: aqi_conversion(x, 'SO2'))
future_beijing['Predicted_AQI_NO2'] = future_beijing['Predicted_NO2'].apply(lambda x: aqi_conversion(x, 'NO2'))
future_beijing['Predicted_AQI_CO'] = future_beijing['Predicted_CO'].apply(lambda x: aqi_conversion(x, 'CO'))
future_beijing['Predicted_AQI_O3'] = future_beijing['Predicted_O3'].apply(lambda x: aqi_conversion(x, 'O3'))

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')][['date', ' pm25', ' pm10', ' so2', ' no2', ' co', ' o3']].reset_index(drop=True)
actual_seoul.columns = ['date', 'Actual_PM25', 'Actual_PM10', 'Actual_SO2', 'Actual_NO2', 'Actual_CO', 'Actual_O3']

# 실제 데이터를 AQI로 변환
actual_seoul['Actual_AQI_PM25'] = actual_seoul['Actual_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
actual_seoul['Actual_AQI_PM10'] = actual_seoul['Actual_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))
actual_seoul['Actual_AQI_SO2'] = actual_seoul['Actual_SO2'].apply(lambda x: aqi_conversion(x, 'SO2'))
actual_seoul['Actual_AQI_NO2'] = actual_seoul['Actual_NO2'].apply(lambda x: aqi_conversion(x, 'NO2'))
actual_seoul['Actual_AQI_CO'] = actual_seoul['Actual_CO'].apply(lambda x: aqi_conversion(x, 'CO'))
actual_seoul['Actual_AQI_O3'] = actual_seoul['Actual_O3'].apply(lambda x: aqi_conversion(x, 'O3'))

# 오차 비율 계산
combined_data = pd.merge(future_beijing, actual_seoul, on='date', how='left')
combined_data['PM25_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM25'] - combined_data['Predicted_AQI_PM25']) / combined_data['Actual_AQI_PM25']) * 100
combined_data['PM10_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM10'] - combined_data['Predicted_AQI_PM10']) / combined_data['Actual_AQI_PM10']) * 100
combined_data['SO2_Error_Rate'] = np.abs((combined_data['Actual_AQI_SO2'] - combined_data['Predicted_AQI_SO2']) / combined_data['Actual_AQI_SO2']) * 100
combined_data['NO2_Error_Rate'] = np.abs((combined_data['Actual_AQI_NO2'] - combined_data['Predicted_AQI_NO2']) / combined_data['Actual_AQI_NO2']) * 100
combined_data['CO_Error_Rate'] = np.abs((combined_data['Actual_AQI_CO'] - combined_data['Predicted_AQI_CO']) / combined_data['Actual_AQI_CO']) * 100
combined_data['O3_Error_Rate'] = np.abs((combined_data['Actual_AQI_O3'] - combined_data['Predicted_AQI_O3']) / combined_data['Actual_AQI_O3']) * 100

# 함수 정의: AQI 값을 입력받아 범주를 반환하는 함수
def categorize_aqi(aqi_value):
    if aqi_value <= 50:
        return 1
    elif aqi_value <= 100:
        return 2
    elif aqi_value <= 150:
        return 3
    elif aqi_value is None:
        return 0
    else:
        return 4

# 예측된 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_predicted_aqi(df):
    df['Predicted_AQI_PM25_Category'] = df['Predicted_AQI_PM25'].apply(categorize_aqi)
    df['Predicted_AQI_PM10_Category'] = df['Predicted_AQI_PM10'].apply(categorize_aqi)
    df['Predicted_AQI_SO2_Category'] = df['Predicted_AQI_SO2'].apply(categorize_aqi)
    df['Predicted_AQI_NO2_Category'] = df['Predicted_AQI_NO2'].apply(categorize_aqi)
    df['Predicted_AQI_CO_Category'] = df['Predicted_AQI_CO'].apply(categorize_aqi)
    df['Predicted_AQI_O3_Category'] = df['Predicted_AQI_O3'].apply(categorize_aqi)
    return df

# 실제 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_actual_aqi(df):
    df['Actual_AQI_PM25_Category'] = df['Actual_AQI_PM25'].apply(categorize_aqi)
    df['Actual_AQI_PM10_Category'] = df['Actual_AQI_PM10'].apply(categorize_aqi)
    df['Actual_AQI_SO2_Category'] = df['Actual_AQI_SO2'].apply(categorize_aqi)
    df['Actual_AQI_NO2_Category'] = df['Actual_AQI_NO2'].apply(categorize_aqi)
    df['Actual_AQI_CO_Category'] = df['Actual_AQI_CO'].apply(categorize_aqi)
    df['Actual_AQI_O3_Category'] = df['Actual_AQI_O3'].apply(categorize_aqi)
    return df

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories(row):
    if row['Predicted_AQI_PM25_Category'] != row['Actual_AQI_PM25_Category']:
        row['PM25_Category_Difference'] = 1
    else:
        row['PM25_Category_Difference'] = 0
    
    if row['Predicted_AQI_PM10_Category'] != row['Actual_AQI_PM10_Category']:
        row['PM10_Category_Difference'] = 1
    else:
        row['PM10_Category_Difference'] = 0

    if row['Predicted_AQI_SO2_Category'] != row['Actual_AQI_SO2_Category']:
        row['SO2_Category_Difference'] = 1
    else:
        row['SO2_Category_Difference'] = 0
    
    if row['Predicted_AQI_NO2_Category'] != row['Actual_AQI_NO2_Category']:
        row['NO2_Category_Difference'] = 1
    else:
        row['NO2_Category_Difference'] = 0

    if row['Predicted_AQI_CO_Category'] != row['Actual_AQI_CO_Category']:
        row['CO_Category_Difference'] = 1
    else:
        row['CO_Category_Difference'] = 0

    if row['Predicted_AQI_O3_Category'] != row['Actual_AQI_O3_Category']:
        row['O3_Category_Difference'] = 1
    else:
        row['O3_Category_Difference'] = 0
    
    return row

# combined_data에 함수 적용
combined_data_categorized = categorize_predicted_aqi(combined_data)
combined_data_categorized = categorize_actual_aqi(combined_data_categorized)

# combined_data_categorized에 함수 적용
combined_data_categorized_diff = combined_data_categorized.apply(compare_aqi_categories, axis=1)

# 결과 출력
print("\n범주화된 예측된 AQI 데이터 및 실제 AQI 데이터:")
# combined_data_categorized_diff[['date', 'Predicted_AQI_PM25_Category', 'Actual_AQI_PM25_Category', 
#                                       'PM25_Category_Difference', 'Predicted_AQI_PM10_Category', 
#                                       'Actual_AQI_PM10_Category', 'PM10_Category_Difference', 
#                                       'Predicted_AQI_SO2_Category', 'Actual_AQI_SO2_Category', 
#                                       'SO2_Category_Difference', 'Predicted_AQI_NO2_Category', 
#                                       'Actual_AQI_NO2_Category', 'NO2_Category_Difference',
#                                       'Predicted_AQI_CO_Category', 'Actual_AQI_CO_Category', 
#                                       'CO_Category_Difference', 'Predicted_AQI_O3_Category', 
#                                       'Actual_AQI_O3_Category', 'O3_Category_Difference']]

# 5월과 6월 동안의 데이터 필터링
year_data = combined_data_categorized_diff[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                               (combined_data_categorized_diff['date'] <= '2024-06-30')]

# PM2.5, PM10, SO2, NO2, CO 및 O3의 카테고리가 같은 비율 계산
total_days = len(year_data)
same_pm25_category_count = year_data[year_data['PM25_Category_Difference'] == 0].shape[0]
same_pm10_category_count = year_data[year_data['PM10_Category_Difference'] == 0].shape[0]
same_so2_category_count = year_data[year_data['SO2_Category_Difference'] == 0].shape[0]
same_no2_category_count = year_data[year_data['NO2_Category_Difference'] == 0].shape[0]
same_co_category_count = year_data[year_data['CO_Category_Difference'] == 0].shape[0]
same_o3_category_count = year_data[year_data['O3_Category_Difference'] == 0].shape[0]

percent_same_pm25_category = (same_pm25_category_count / total_days) * 100
percent_same_pm10_category = (same_pm10_category_count / total_days) * 100
percent_same_so2_category = (same_so2_category_count / total_days) * 100
percent_same_no2_category = (same_no2_category_count / total_days) * 100
percent_same_co_category = (same_co_category_count / total_days) * 100
percent_same_o3_category = (same_o3_category_count / total_days) * 100

# 결과 출력
print(f"\n2024년 동안 PM2.5의 예측 카테고리가 실제와 같은 비율: {percent_same_pm25_category:.2f}%")
print(f"2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율: {percent_same_pm10_category:.2f}%")
print(f"2024년 동안 SO2의 예측 카테고리가 실제와 같은 비율: {percent_same_so2_category:.2f}%")
print(f"2024년 동안 NO2의 예측 카테고리가 실제와 같은 비율: {percent_same_no2_category:.2f}%")
print(f"2024년 동안 CO의 예측 카테고리가 실제와 같은 비율: {percent_same_co_category:.2f}%")
print(f"2024년 동안 O3의 예측 카테고리가 실제와 같은 비율: {percent_same_o3_category:.2f}%")


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
-  co_seoul
-  no2_seoul
-  o3_seoul
-  pm10_seoul
-  so2_seoul


,date,pm25,pm10,o3,no2,so2,co
0,2024-07-01,68,18,29,14,3,4
1,2024-07-02,42,29,51,10,3,5
2,2024-07-03,59,44,75,11,3,6
3,2024-07-04,108,17,51,6,3,4
4,2024-07-05,33,31,42,13,3,5
...,...,...,...,...,...,...,...
3773,2014-03-28,,93,19,49,8,6
3774,2014-03-29,,53,43,25,7,5
3775,2014-03-30,,38,22,64,8,8
3776,2014-03-31,,61,26,77,9,8


In [90]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression

# 데이터 불러오기
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 데이터를 필터링
seoul_past = seoul[seoul['date'] < '2024-01-01']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 서울 데이터프레임을 숫자로 변환하여 처리
seoul_numeric = preprocess(seoul_past)

# 예측 모델을 위한 준비
def prepare_data(df, target):
    X = df.drop(columns=[target])
    y = df[target]
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.mean(), inplace=True)
    return X, y

# 모델 생성 및 학습
def create_and_train_model(df, target):
    X, y = prepare_data(df, target)
    model = LinearRegression()
    model.fit(X, y)
    return model, X.columns

# 결측치 처리
seoul_numeric.fillna(seoul_numeric.mean(), inplace=True)
seoul_numeric.fillna(0, inplace=True)

# 예측 모델 생성
model_pm25, features_pm25 = create_and_train_model(seoul_numeric, ' pm25')
model_pm10, features_pm10 = create_and_train_model(seoul_numeric, ' pm10')
model_so2, features_so2 = create_and_train_model(seoul_numeric, ' so2')
model_no2, features_no2 = create_and_train_model(seoul_numeric, ' no2')
model_co, features_co = create_and_train_model(seoul_numeric, ' co')
model_o3, features_o3 = create_and_train_model(seoul_numeric, ' o3')

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_seoul = pd.DataFrame({
    'date': future_dates,
    ' pm25': [random.randint(50, 150) for _ in range(len(future_dates))],
    ' pm10': [random.randint(30, 100) for _ in range(len(future_dates))],
    ' so2': [random.randint(5, 25) for _ in range(len(future_dates))],
    ' no2': [random.randint(20, 70) for _ in range(len(future_dates))],
    ' co': [random.uniform(0.5, 2.0) for _ in range(len(future_dates))],
    ' o3': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# 예측 데이터 준비
def predict_future(model, future_data, feature_names):
    future_data = future_data[feature_names]
    return model.predict(future_data)

# 예측 수행
future_X = future_seoul.drop(columns=['date'])
future_seoul['Predicted_PM25'] = predict_future(model_pm25, future_X, features_pm25)
future_seoul['Predicted_PM10'] = predict_future(model_pm10, future_X, features_pm10)
future_seoul['Predicted_SO2'] = predict_future(model_so2, future_X, features_so2)
future_seoul['Predicted_NO2'] = predict_future(model_no2, future_X, features_no2)
future_seoul['Predicted_CO'] = predict_future(model_co, future_X, features_co)
future_seoul['Predicted_O3'] = predict_future(model_o3, future_X, features_o3)


        
# 결과 출력
future_seoul

,date,pm25,pm10,so2,no2,co,o3,Predicted_PM25,Predicted_PM10,Predicted_SO2,Predicted_NO2,Predicted_CO,Predicted_O3
0,2024-01-01,149,32,13,38,1.743401,68,64.647532,67.980522,6.126623,22.412296,6.107939,35.812986
1,2024-01-02,121,41,8,69,0.619270,59,60.642016,54.078755,9.324692,11.782828,9.497250,32.355956
2,2024-01-03,69,43,24,63,0.875735,52,66.461355,64.501360,8.726729,35.938739,8.358269,27.475270
3,2024-01-04,65,62,5,27,1.690433,43,73.262804,29.413104,5.584806,13.678937,5.196216,43.230736
4,2024-01-05,137,91,22,36,1.681713,66,96.157116,77.538384,6.970520,36.811813,6.583625,45.740240
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2024-12-27,97,39,21,22,1.220479,79,70.140862,67.919873,4.595050,31.101126,3.785972,38.424928
362,2024-12-28,57,78,9,61,0.869219,64,81.458213,43.384278,9.081528,15.080224,8.554807,38.323477
363,2024-12-29,103,56,8,54,1.026719,42,67.524947,43.892093,8.174235,15.068155,8.265431,37.527228
364,2024-12-30,146,91,7,33,1.699475,55,90.161789,53.614478,6.741723,16.509586,6.723398,50.579248


In [88]:
def calculate_aqi_o3(concentration):
    # AQI breakpoints and corresponding AQI values for O3 (8-hour average)
    breakpoints = [
        (0.000, 0.054, 0, 50),
        (0.055, 0.070, 51, 100),
        (0.071, 0.085, 101, 150),
        (0.086, 0.105, 151, 200),
        (0.106, 0.200, 201, 300),
        (0.201, 0.400, 301, 400),
    ]
    
    # Initialize variables
    C_low = 0
    C_high = 0
    I_low = 0
    I_high = 0
    
    # Determine which breakpoint range the concentration falls into
    for bp in breakpoints:
        if bp[0] <= concentration <= bp[1]:
            C_low, C_high = bp[0], bp[1]
            I_low, I_high = bp[2], bp[3]
            break
    else:
        raise ValueError("Concentration is out of range for O3 AQI calculation")
    
    # Calculate AQI using the EPA formula
    if C_high - C_low == 0:
        raise ValueError("Division by zero error: C_high - C_low == 0")
    
    aqi = ((I_high - I_low) * (concentration - C_low) / (C_high - C_low)) + I_low
    
    return round(aqi)

# Example: Convert 0.03 ppm to AQI for O3
ppm_to_ugm3 = 0.03 * 1000  # convert ppm to μg/m³ (1 ppm = 1000 μg/m³)
try:
    aqi_result = calculate_aqi_o3(ppm_to_ugm3)
    print(f"0.03 ppm의 오존 농도는 약 {aqi_result} AQI입니다.")
except ValueError as e:
    print(f"오류 발생: {e}")


오류 발생: Concentration is out of range for O3 AQI calculation


TypeError: Could not convert [' 22 25 50 53 44 46 47 36 34 64 86 111 100 75 90 136 137 56 44 34 49 39 78 57 80 96 63 68 42 28 32 57 50 29 36 68 97 60 66 51 51 34 64 73 57 72 56 74 61 71 81 43 64 62 48 46 62 58 52 67 64 45 46 34 47 43 39 41 32 60 54 48 47 48 40 37 68 86 63 33 47 46 47 47 38 46 46 31 31 20 55 64 49 46 28 38 19 40 86 133 176 158 138 67 38 62 106 63 140 73 91 64 45 98 75 87 90 39 68 47 56 46 62 120 87 87 124 97 126 47 62 78 77 64 63 57 83 157 189 143 172 146 102 42 49 40 17 48 48 41 53 66 67 111 78 122 99 105 35 102 156 111 88 77 101 99 60 54 63 77 99 135 131 54 59 67 82 46 69 97 84 80 101 125 117 83 123 102 62 28 31 53 61 21 43 61 49 49 50 46 50 55 49 57 38 43 81 48 17 32 57 70 90 98 89 39 68 47 56 74 129 82 34 36 53 75 56 66 77 61 49 40 41 89 125 118 28 63 89 29 83 89 115 23 35 58 105 32 53 43 70 98 45 122 176 148 90 118 106 66 53 75 93 49 65 61 48 94 128 48 53 63 81 143 158 158 65 125 169 151 156 50 31 37 42 56 76 52 20 57 58 49 44 40 58 67 35 37 31 64 62 62 30 49 66 69

In [157]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))
corr_beijing_seoul.fillna(corr_beijing_seoul.mean(), inplace=True)
corr_beijing_seoul.fillna(0, inplace=True)
corr_beijing_seoul.columns = corr_beijing_seoul.columns.str.replace(' ', '')
# PM2.5 예측 모델
X_pm25 = corr_beijing_seoul[['pm25_beijing', 'pm10_beijing', 'o3_beijing', 'no2_beijing', 'so2_beijing', 'co_beijing', 
                             'pm10_seoul', 'o3_seoul', 'no2_seoul', 'so2_seoul', 'co_seoul']]
y_pm25 = corr_beijing_seoul['pm25_seoul']

# PM10 예측 모델
X_pm10 = corr_beijing_seoul[['pm25_beijing', 'pm10_beijing', 'o3_beijing', 'no2_beijing', 'so2_beijing', 'co_beijing', 
                             'pm25_seoul', 'o3_seoul', 'no2_seoul', 'so2_seoul', 'co_seoul']]
y_pm10 = corr_beijing_seoul['pm10_seoul']

# o3 예측 모델
X_o3 = corr_beijing_seoul[['pm25_beijing', 'pm10_beijing', 'o3_beijing', 'no2_beijing', 'so2_beijing', 'co_beijing', 
                            'pm25_seoul', 'pm10_seoul', 'no2_seoul', 'so2_seoul', 'co_seoul']]
y_o3 = corr_beijing_seoul['o3_seoul']

# no2 예측 모델
X_no2 = corr_beijing_seoul[['pm25_beijing', 'pm10_beijing', 'o3_beijing', 'no2_beijing', 'so2_beijing', 'co_beijing', 
                             'pm25_seoul', 'o3_seoul', 'pm10_seoul', 'so2_seoul', 'co_seoul']]
y_no2 = corr_beijing_seoul['no2_seoul']

# so2 예측 모델
X_so2 = corr_beijing_seoul[['pm25_beijing', 'pm10_beijing', 'o3_beijing', 'no2_beijing', 'so2_beijing', 'co_beijing', 
                             'pm25_seoul', 'o3_seoul', 'no2_seoul', 'pm10_seoul', 'co_seoul']]
y_so2 = corr_beijing_seoul['so2_seoul']

# co 예측 모델
X_co = corr_beijing_seoul[['pm25_beijing', 'pm10_beijing', 'o3_beijing', 'no2_beijing', 'so2_beijing', 'co_beijing', 
                             'pm25_seoul', 'o3_seoul', 'no2_seoul', 'so2_seoul', 'pm10_seoul']]
y_co = corr_beijing_seoul['co_seoul']

# 데이터 준비 함수
def prepare_data(X, y):
    X = X.copy()
    y = y.copy()
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.astype(float).mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.astype(float).mean(), inplace=True)
    return X.astype(float), y.astype(float)

# 데이터 준비
X_pm25, y_pm25 = prepare_data(X_pm25, y_pm25)
X_pm10, y_pm10 = prepare_data(X_pm10, y_pm10)
X_o3, y_o3 = prepare_data(X_o3, y_o3)
X_no2, y_no2 = prepare_data(X_no2, y_no2)
X_so2, y_so2 = prepare_data(X_so2, y_so2)
X_co, y_co = prepare_data(X_co, y_co)

# 모델 생성 및 학습
model_pm25 = LinearRegression()
model_pm25.fit(X_pm25, y_pm25)

model_pm10 = LinearRegression()
model_pm10.fit(X_pm10, y_pm10)
 
model_o3 = LinearRegression()
model_o3.fit(X_o3, y_o3)

model_no2 = LinearRegression()
model_no2.fit(X_no2, y_no2)
 
model_so2 = LinearRegression()
model_so2.fit(X_so2, y_so2)

model_co = LinearRegression()
model_co.fit(X_co, y_co)
 
# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates,
    'pm25': [random.randint(50, 150) for _ in range(len(future_dates))],
    'pm10': [random.randint(30, 100) for _ in range(len(future_dates))],
    'o3': [random.randint(30, 100) for _ in range(len(future_dates))],
    'no2': [random.randint(30, 100) for _ in range(len(future_dates))],
    'so2': [random.randint(1, 30) for _ in range(len(future_dates))],
    'co': [random.randint(1, 10) for _ in range(len(future_dates))]
})

# PM2.5, PM10 예측
future_X_pm25 = future_beijing[['pm25', 'pm10', 'o3', 'no2', 'so2', 'co']]
future_X_pm10 = future_beijing[['pm25', 'pm10', 'o3', 'no2', 'so2', 'co']]
future_X_o3 = future_beijing[['pm25', 'pm10', 'o3', 'no2', 'so2', 'co']]
future_X_no2 = future_beijing[['pm25', 'pm10', 'o3', 'no2', 'so2', 'co']]
future_X_so2 = future_beijing[['pm25', 'pm10', 'o3', 'no2', 'so2', 'co']]
future_X_co = future_beijing[['pm25', 'pm10', 'o3', 'no2', 'so2', 'co']]

future_pred_pm25 = model_pm25.predict(future_X_pm25)
future_pred_pm10 = model_pm10.predict(future_X_pm10)
future_pred_o3 = model_o3.predict(future_X_o3)
future_pred_no2 = model_no2.predict(future_X_no2)
future_pred_so2 = model_so2.predict(future_X_so2)
future_pred_co = model_co.predict(future_X_co)

future_beijing['Predicted_PM25'] = future_pred_pm25
future_beijing['Predicted_PM10'] = future_pred_pm10
future_beijing['Predicted_o3'] = future_pred_o3
future_beijing['Predicted_no2'] = future_pred_no2
future_beijing['Predicted_so2'] = future_pred_so2
future_beijing['Predicted_co'] = future_pred_co

# 예측된 AQI 계산 함수
def aqi_conversion(df):
    df['AQI'] = (((df['Predicted_PM25']/35.5)+(df['Predicted_PM10']/50.0)+(df['Predicted_o3']/0.125)+(df['Predicted_no2']/0.100)+(df['Predicted_so2']/0.060)+(df['Predicted_co']/0.500))/6)*100
    return df

future_beijing_aqi = aqi_conversion(future_beijing)

# 예측된 AQI 범주화 함수
def categorize_predicted_aqi(df):
    bins = [0, 50, 100, 150, 200, 300, 500]
    labels = ['Good', 'Moderate', 'Unhealthy for Sensitive Groups', 'Unhealthy', 'Very Unhealthy', 'Hazardous']
    df['AQI_Category'] = pd.cut(df['AQI'], bins=bins, labels=labels)
    return df

future_beijing_aqi_categorized = categorize_predicted_aqi(future_beijing_aqi)

# 실제 AQI 범주화 함수
def categorize_actual_aqi(df):
    bins = [0, 50, 100, 150, 200, 300, 500]
    labels = ['Good', 'Moderate', 'Unhealthy for Sensitive Groups', 'Unhealthy', 'Very Unhealthy', 'Hazardous']
    df['Actual_AQI_Category'] = pd.cut(df['AQI'], bins=bins, labels=labels)
    return df

# 실제 AQI 데이터프레임 준비
actual_aqi_data = future_beijing_aqi.copy()  # 예제 데이터 사용, 실제 데이터로 교체 필요

# AQI 카테고리 비교 함수
def compare_aqi_categories(predicted_df, actual_df):
    predicted_df['Actual_AQI_Category'] = categorize_actual_aqi(actual_df)['Actual_AQI_Category']
    match_count = (predicted_df['AQI_Category'] == predicted_df['Actual_AQI_Category']).sum()
    total_count = len(predicted_df)
    accuracy = match_count / total_count * 100
    return accuracy

# 예측된 AQI와 실제 AQI 비교
accuracy = compare_aqi_categories(future_beijing_aqi_categorized, actual_aqi_data)
print(f"2024년에 대한 PM2.5 및 PM10 예측 정확도: {accuracy:.2f}%")

# 예측된 AQI 데이터 출력 (예시로 첫 번째 행만 출력)
print(future_beijing_aqi_categorized.head(1))


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- co
- no2
- o3
- pm10
- pm25
- ...
Feature names seen at fit time, yet now missing:
- co_beijing
- co_seoul
- no2_beijing
- no2_seoul
- o3_beijing
- ...


In [152]:
seoul.columns.tolist()

['date', ' pm25', ' pm10', ' o3', ' no2', ' so2', ' co']

In [158]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

# PM2.5 예측 모델
X_pm25 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm25 = corr_beijing_seoul[' pm25_seoul']

# PM10 예측 모델
X_pm10 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm10 = corr_beijing_seoul[' pm10_seoul']

# 데이터 준비 함수
def prepare_data(X, y):
    X = X.copy()
    y = y.copy()
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.astype(float).mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.astype(float).mean(), inplace=True)
    return X.astype(float), y.astype(float)

# PM2.5, PM10 데이터 준비
X_pm25, y_pm25 = prepare_data(X_pm25, y_pm25)
X_pm10, y_pm10 = prepare_data(X_pm10, y_pm10)

# 모델 생성 및 학습
model_pm25 = LinearRegression()
model_pm25.fit(X_pm25, y_pm25)

model_pm10 = LinearRegression()
model_pm10.fit(X_pm10, y_pm10)

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates,
    ' pm25_beijing': [random.randint(50, 150) for _ in range(len(future_dates))],
    ' pm10_beijing': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# PM2.5, PM10 예측
future_X_pm25 = future_beijing[[' pm25_beijing', ' pm10_beijing']]
future_X_pm10 = future_beijing[[' pm25_beijing', ' pm10_beijing']]

future_pred_pm25 = model_pm25.predict(future_X_pm25)
future_pred_pm10 = model_pm10.predict(future_X_pm10)

future_beijing['Predicted_PM25'] = future_pred_pm25
future_beijing['Predicted_PM10'] = future_pred_pm10

# AQI를 μg/m³로 변환하는 함수 정의 (한국 기준)
def aqi_conversion(pm_value, pm_type):
    if pm_type == 'PM2.5':
        if pm_value <= 12:
            return pm_value * (15.0 / 12)
        elif pm_value <= 35.4:
            return 15.0 + (pm_value - 12) * ((35.0 - 15.0) / (35.4 - 12))
        elif pm_value <= 75.4:
            return 35.0 + (pm_value - 35.4) * ((75.0 - 35.0) / (75.4 - 35.4))
        elif pm_value <= 115.4:
            return 75.0 + (pm_value - 75.4) * ((115.0 - 75.0) / (115.4 - 75.4))
        elif pm_value <= 150.4:
            return 115.0 + (pm_value - 115.4) * ((150.0 - 115.0) / (150.4 - 115.4))
        elif pm_value <= 500:
            return 150.0 + (pm_value - 150.4) * ((350.0 - 150.0) / (500 - 150.4))
        else:
            return 500.0
    elif pm_type == 'PM10':
        if pm_value <= 30:
            return pm_value * (30.0 / 30)
        elif pm_value <= 80:
            return 30.0 + (pm_value - 30) * ((80.0 - 30.0) / (80 - 30))
        elif pm_value <= 150:
            return 80.0 + (pm_value - 80) * ((150.0 - 80.0) / (150 - 80))
        elif pm_value <= 250:
            return 150.0 + (pm_value - 150) * ((250.0 - 150.0) / (250 - 150))
        elif pm_value <= 350:
            return 250.0 + (pm_value - 250) * ((350.0 - 250.0) / (350 - 250))
        elif pm_value <= 420:
            return 350.0 + (pm_value - 350) * ((420.0 - 350.0) / (420 - 350))
        else:
            return 420.0

# 예측 데이터를 AQI로 변환
future_beijing['Predicted_AQI_PM25'] = future_beijing['Predicted_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
future_beijing['Predicted_AQI_PM10'] = future_beijing['Predicted_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')][['date', ' pm25', ' pm10']].reset_index(drop=True)
actual_seoul.columns = ['date', 'Actual_PM25', 'Actual_PM10']

# 'Actual_PM25'와 'Actual_PM10' 열을 숫자로 변환
actual_seoul['Actual_PM25'] = pd.to_numeric(actual_seoul['Actual_PM25'], errors='coerce')
actual_seoul['Actual_PM10'] = pd.to_numeric(actual_seoul['Actual_PM10'], errors='coerce')

# 데이터를 AQI로 변환
actual_seoul['Actual_AQI_PM25'] = actual_seoul['Actual_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
actual_seoul['Actual_AQI_PM10'] = actual_seoul['Actual_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))

# 오차 비율 계산
combined_data = pd.merge(future_beijing, actual_seoul, on='date', how='left')
combined_data['PM25_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM25'] - combined_data['Predicted_AQI_PM25']) / combined_data['Actual_AQI_PM25']) * 100
combined_data['PM10_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM10'] - combined_data['Predicted_AQI_PM10']) / combined_data['Actual_AQI_PM10']) * 100

# 함수 정의: AQI 값을 입력받아 범주를 반환하는 함수
def categorize_aqi(aqi_value):
    if aqi_value <= 50:
        return 1
    elif aqi_value <= 100:
        return 2
    elif aqi_value <= 150:
        return 3
    else:
        return 4

# 예측된 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_predicted_aqi(df):
    df['Predicted_AQI_PM25_Category'] = df['Predicted_AQI_PM25'].apply(categorize_aqi)
    df['Predicted_AQI_PM10_Category'] = df['Predicted_AQI_PM10'].apply(categorize_aqi)
    return df

# 실제 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_actual_aqi(df):
    df['Actual_AQI_PM25_Category'] = df['Actual_AQI_PM25'].apply(categorize_aqi)
    df['Actual_AQI_PM10_Category'] = df['Actual_AQI_PM10'].apply(categorize_aqi)
    return df

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories(row):
    if row['Predicted_AQI_PM25_Category'] != row['Actual_AQI_PM25_Category']:
        row['PM25_Category_Difference'] = 1
    else:
        row['PM25_Category_Difference'] = 0
    
    if row['Predicted_AQI_PM10_Category'] != row['Actual_AQI_PM10_Category']:
        row['PM10_Category_Difference'] = 1
    else:
        row['PM10_Category_Difference'] = 0
    
    return row

# COMBINED_DATA에 적용
combined_data_categorized = categorize_predicted_aqi(combined_data)
combined_data_categorized = categorize_actual_aqi(combined_data_categorized)

# combined_data_categorized에 함수 적용
combined_data_categorized_diff = combined_data_categorized.apply(compare_aqi_categories, axis=1)

# 결과 출력
print("2024년 미세먼지 예측 적중률")
# combined_data_categorized_diff[['date', 'Predicted_AQI_PM25_Category', 'Actual_AQI_PM25_Category', 
#                                       'PM25_Category_Difference', 'Predicted_AQI_PM10_Category', 
#                                       'Actual_AQI_PM10_Category', 'PM10_Category_Difference']]

# 2024년 한 해 동안의 데이터 필터링
total_data = combined_data_categorized_diff[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-12-31')]

year_data = total_data[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-06-30')]

# PM2.5 및 PM10의 카테고리가 같은 비율 계산
total_days = len(year_data)
same_pm25_category_count = year_data[year_data['PM25_Category_Difference'] == 0].shape[0]
same_pm10_category_count = year_data[year_data['PM10_Category_Difference'] == 0].shape[0]

percent_same_pm25_category = (same_pm25_category_count / total_days) * 100
percent_same_pm10_category = (same_pm10_category_count / total_days) * 100

# 결과 출력
print(f"2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율: {percent_same_pm25_category:.2f}%")
print(f"2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율: {percent_same_pm10_category:.2f}%\n")

# 특정 날짜의 데이터 출력 함수
def print_aqi_for_date(target_date):
    target_date = pd.to_datetime(target_date)
    row = combined_data[combined_data['date'] == target_date]
    if row.empty:
        print(f"No data available for the date: {target_date.strftime('%Y-%m-%d')}")
    else:
        print(f"{target_date.strftime('%Y-%m-%d')} 대기질 예측 데이터")
        print(f"예상 PM25 : {round(row['Predicted_AQI_PM25'].values[0])} (Category: {row['Predicted_AQI_PM25_Category'].values[0]})")
        print(f"예상 PM10 : {round(row['Predicted_AQI_PM10'].values[0])} (Category: {row['Predicted_AQI_PM10_Category'].values[0]})")

# 특정 날짜의 AQI 데이터를 출력
target_date = '2024-09-30'
print_aqi_for_date(target_date)

2024년 미세먼지 예측 적중률
2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율: 60.44%
2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율: 72.53%

2024-09-30 대기질 예측 데이터
예상 PM25 : 76 (Category: 2)
예상 PM10 : 47 (Category: 1)


In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

corr_beijing_seoul.fillna(corr_beijing_seoul.mean(), inplace=True)
corr_beijing_seoul.fillna(0, inplace=True)

# O3 예측 모델
X_o3 = corr_beijing_seoul[[' o3_beijing', ' co_beijing']]
y_o3 = corr_beijing_seoul[' o3_seoul']

# CO 예측 모델
X_co = corr_beijing_seoul[[' o3_beijing', ' co_beijing']]
y_co = corr_beijing_seoul[' co_seoul']

# 데이터 준비 함수
def prepare_data(X, y):
    X = X.copy()
    y = y.copy()
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.astype(float).mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.astype(float).mean(), inplace=True)
    return X.astype(float), y.astype(float)

# O3, CO 데이터 준비
X_o3, y_o3 = prepare_data(X_o3, y_o3)
X_co, y_co = prepare_data(X_co, y_co)

# 모델 생성 및 학습
model_o3 = LinearRegression()
model_o3.fit(X_o3, y_o3)

model_co = LinearRegression()
model_co.fit(X_co, y_co)

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates,
    ' o3_beijing': [random.randint(50, 150) for _ in range(len(future_dates))],
    ' co_beijing': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# O3, CO 예측
future_X_o3 = future_beijing[[' o3_beijing', ' co_beijing']]
future_X_co = future_beijing[[' o3_beijing', ' co_beijing']]

future_pred_o3 = model_o3.predict(future_X_o3)
future_pred_co = model_co.predict(future_X_co)

future_beijing['Predicted_O3'] = future_pred_o3
future_beijing['Predicted_CO'] = future_pred_co

# AQI를 μg/m³로 변환하는 함수 정의 (한국 기준)
def aqi_conversion(pm_value, pm_type):
    if pm_type == 'O3':
        if pm_value <= 0.03:
            return pm_value * (50.0 / 0.03)
        elif pm_value <= 0.09:
            return 50.0 + (pm_value - 0.03) * ((100.0 - 50.0) / (0.09 - 0.03))
        elif pm_value <= 0.15:
            return 100.0 + (pm_value - 0.09) * ((150.0 - 100.0) / (0.15 - 0.09))
        elif pm_value <= 0.60:
            return 150.0 + (pm_value - 0.15) * ((200.0 - 150.0) / (0.60 - 0.15))
        elif pm_value <= 2.00:
            return 200.0 + (pm_value - 0.60) * ((300.0 - 200.0) / (2.00 - 0.60))
        else:
            return 300.0
    elif pm_type == 'CO':
        if pm_value <= 2.0:
            return pm_value * (50.0 / 2.0)
        elif pm_value <= 9.0:
            return 50.0 + (pm_value - 2.0) * ((100.0 - 50.0) / (9.0 - 2.0))
        elif pm_value <= 15.0:
            return 100.0 + (pm_value - 9.0) * ((150.0 - 100.0) / (15.0 - 9.0))
        elif pm_value <= 30.0:
            return 150.0 + (pm_value - 15.0) * ((200.0 - 150.0) / (30.0 - 15.0))
        elif pm_value <= 40.0:
            return 200.0 + (pm_value - 30.0) * ((300.0 - 200.0) / (40.0 - 30.0))
        else:
            return 300.0

# 예측 데이터를 AQI로 변환
future_beijing['Predicted_AQI_O3'] = future_beijing['Predicted_O3'].apply(lambda x: aqi_conversion(x, 'O3'))
future_beijing['Predicted_AQI_CO'] = future_beijing['Predicted_CO'].apply(lambda x: aqi_conversion(x, 'CO'))

# 예측 데이터 출력
print("2024년 O3 및 CO 예측 데이터")
print(future_beijing[['date', 'Predicted_O3', 'Predicted_AQI_O3', 'Predicted_CO', 'Predicted_AQI_CO']])

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')][['date', ' o3', ' co']].reset_index(drop=True)
actual_seoul.columns = ['date', 'Actual_O3', 'Actual_CO']

# 'Actual_O3'와 'Actual_CO' 열을 숫자로 변환
actual_seoul['Actual_O3'] = pd.to_numeric(actual_seoul['Actual_O3'], errors='coerce')
actual_seoul['Actual_CO'] = pd.to_numeric(actual_seoul['Actual_CO'], errors='coerce')

# 데이터를 AQI로 변환
actual_seoul['Actual_AQI_O3'] = actual_seoul['Actual_O3'].apply(lambda x: aqi_conversion(x, 'O3'))
actual_seoul['Actual_AQI_CO'] = actual_seoul['Actual_CO'].apply(lambda x: aqi_conversion(x, 'CO'))

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories(row):
    if row['Predicted_AQI_O3'] <= 50:
        row['Predicted_AQI_O3_Category'] = 1
    elif row['Predicted_AQI_O3'] <= 100:
        row['Predicted_AQI_O3_Category'] = 2
    elif row['Predicted_AQI_O3'] <= 150:
        row['Predicted_AQI_O3_Category'] = 3
    else:
        row['Predicted_AQI_O3_Category'] = 4
    
    if row['Predicted_AQI_CO'] <= 50:
        row['Predicted_AQI_CO_Category'] = 1
    elif row['Predicted_AQI_CO'] <= 100:
        row['Predicted_AQI_CO_Category'] = 2
    elif row['Predicted_AQI_CO'] <= 150:
        row['Predicted_AQI_CO_Category'] = 3
    else:
        row['Predicted_AQI_CO_Category'] = 4
    
    if row['Actual_AQI_O3'] <= 50:
        row['Actual_AQI_O3_Category'] = 1
    elif row['Actual_AQI_O3'] <= 100:
        row['Actual_AQI_O3_Category'] = 2
    elif row['Actual_AQI_O3'] <= 150:
        row['Actual_AQI_O3_Category'] = 3
    else:
        row['Actual_AQI_O3_Category'] = 4
    
    if row['Actual_AQI_CO'] <= 50:
        row['Actual_AQI_CO_Category'] = 1
    elif row['Actual_AQI_CO'] <= 100:
        row['Actual_AQI_CO_Category'] = 2
    elif row['Actual_AQI_CO'] <= 150:
        row['Actual_AQI_CO_Category'] = 3
    else:
        row['Actual_AQI_CO_Category'] = 4
    
    if row['Predicted_AQI_O3_Category'] != row['Actual_AQI_O3_Category'] or row['Predicted_AQI_CO_Category'] != row['Actual_AQI_CO_Category']:
        return 1
    else:
        return 0

# 정확도 계산
accuracy = 1 - (actual_seoul.apply(compare_aqi_categories, axis=1).sum() / len(actual_seoul))

# 정확도 출력
print(f"\n2024년 예측 정확도: {accuracy:.2%}")

# 특정 날짜 데이터 출력 예시 (첫 번째 데이터)
print("\n2024년 1월 1일 데이터 예시:")
print(actual_seoul.head(1))

In [203]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    numeric_df.rename(columns={' pm25': 'co', ' pm10': 'o3'}, inplace=True)  # Rename columns
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

# CO 예측 모델
X_co = corr_beijing_seoul[['co_beijing', 'o3_beijing']]
y_co = corr_beijing_seoul['co_seoul']

# O3 예측 모델
X_o3 = corr_beijing_seoul[['co_beijing', 'o3_beijing']]
y_o3 = corr_beijing_seoul['o3_seoul']

# 데이터 준비 함수
def prepare_data(X, y):
    X = X.copy()
    y = y.copy()
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.astype(float).mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.astype(float).mean(), inplace=True)
    return X.astype(float), y.astype(float)

# CO, O3 데이터 준비
X_co, y_co = prepare_data(X_co, y_co)
X_o3, y_o3 = prepare_data(X_o3, y_o3)

# 모델 생성 및 학습
model_co = LinearRegression()
model_co.fit(X_co, y_co)

model_o3 = LinearRegression()
model_o3.fit(X_o3, y_o3)

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates,
    'co_beijing': [random.randint(50, 150) for _ in range(len(future_dates))],
    'o3_beijing': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# CO, O3 예측
future_X_co = future_beijing[['co_beijing', 'o3_beijing']]
future_X_o3 = future_beijing[['co_beijing', 'o3_beijing']]

future_pred_co = model_co.predict(future_X_co)
future_pred_o3 = model_o3.predict(future_X_o3)

future_beijing['Predicted_CO'] = future_pred_co
future_beijing['Predicted_O3'] = future_pred_o3

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')][['date', ' pm25', ' pm10']].reset_index(drop=True)
actual_seoul.columns = ['date', 'Actual_CO', 'Actual_O3']

# 'Actual_CO'와 'Actual_O3' 열을 숫자로 변환
actual_seoul['Actual_CO'] = pd.to_numeric(actual_seoul['Actual_CO'], errors='coerce')
actual_seoul['Actual_O3'] = pd.to_numeric(actual_seoul['Actual_O3'], errors='coerce')

# 데이터 병합
combined_data = pd.merge(future_beijing, actual_seoul, on='date', how='left')

# O3 예측 오차 범주화 함수 정의
def categorize_o3_error(error):
    if error <= 100:
        return 1
    elif error <= 200:
        return 2
    elif error <= 300:
        return 3
    elif error <= 400:
        return 4
    else:
        return 0

combined_data['CO_Predict_Category'] = combined_data['Predicted_CO'].apply(categorize_o3_error)
combined_data['O3_Predict_Category'] = combined_data['Predicted_O3'].apply(categorize_o3_error)
combined_data['CO_Actual_Category'] = combined_data['Actual_CO'].apply(categorize_o3_error)
combined_data['O3_Actual_Category'] = combined_data['Actual_O3'].apply(categorize_o3_error)

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories(row):
    if row['CO_Predict_Category'] != row['CO_Actual_Category']:
        row['CO_Category_Difference'] = 1
    else:
        row['CO_Category_Difference'] = 0
    
    if row['O3_Predict_Category'] != row['O3_Actual_Category']:
        row['O3_Category_Difference'] = 1
    else:
        row['O3_Category_Difference'] = 0
    
    return row

# COMBINED_DATA에 적용
combined_data_diff = combined_data.apply(compare_aqi_categories, axis=1)

# 2024년 한 해 동안의 데이터 필터링
total_data = combined_data_categorized_diff[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-12-31')]

year_data = total_data[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-06-30')]

# PM2.5 및 PM10의 카테고리가 같은 비율 계산
total_days = len(year_data)
same_co_category_count = year_data[year_data['CO_Category_Difference'] == 0].shape[0]
same_o3_category_count = year_data[year_data['O3_Category_Difference'] == 0].shape[0]

percent_same_co_category = (same_co_category_count / total_days) * 100
percent_same_o3_category = (same_o3_category_count / total_days) * 100

# 결과 출력
print(f"2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율: {percent_same_co_category:.2f}%")
print(f"2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율: {percent_same_o3_category:.2f}%\n")

# 특정 날짜의 AQI 데이터를 출력 함수
def print_aqi_for_date(target_date):
    target_date = pd.to_datetime(target_date)
    row = combined_data[combined_data['date'] == target_date]
    if row.empty:
        print(f"No data available for the date: {target_date.strftime('%Y-%m-%d')}")
    else:
        print(f"{target_date.strftime('%Y-%m-%d')} 대기질 예측 데이터")
        print(f"예상 CO : {round(row['Predicted_CO'].values[0])} (Category: {row['CO_Predict_Category'].values[0]})")
        print(f"예상 O3 : {round(row['Predicted_O3'].values[0])} (Category: {row['O3_Predict_Category'].values[0]})")
        actual_co = round(row['Actual_CO'].values[0]) if not np.isnan(row['Actual_CO'].values[0]) else 'N/A'
        actual_o3 = round(row['Actual_O3'].values[0]) if not np.isnan(row['Actual_O3'].values[0]) else 'N/A'
        actual_co_category = row['CO_Actual_Category'].values[0] if not np.isnan(row['Actual_CO'].values[0]) else 'N/A'
        actual_o3_category = row['O3_Actual_Category'].values[0] if not np.isnan(row['Actual_O3'].values[0]) else 'N/A'
        print(f"실제 CO : {actual_co} (Category: {actual_co_category})")
        print(f"실제 O3 : {actual_o3} (Category: {actual_o3_category})")

# 특정 날짜의 AQI 데이터를 출력
target_date = '2024-12-25'
print_aqi_for_date(target_date)


2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율: 97.80%
2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율: 100.00%

2024-12-25 대기질 예측 데이터
예상 CO : 84 (Category: 1)
예상 O3 : 48 (Category: 1)
실제 CO : N/A (Category: N/A)
실제 O3 : N/A (Category: N/A)


In [ ]:
# 2024년 한 해 동안의 데이터 필터링
total_data = combined_data_categorized_diff[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-12-31')]

year_data = total_data[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-06-30')]

# PM2.5 및 PM10의 카테고리가 같은 비율 계산
total_days = len(year_data)
same_pm25_category_count = year_data[year_data['PM25_Category_Difference'] == 0].shape[0]
same_pm10_category_count = year_data[year_data['PM10_Category_Difference'] == 0].shape[0]

percent_same_pm25_category = (same_pm25_category_count / total_days) * 100
percent_same_pm10_category = (same_pm10_category_count / total_days) * 100

# 결과 출력
print(f"2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율: {percent_same_pm25_category:.2f}%")
print(f"2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율: {percent_same_pm10_category:.2f}%\n")

# 특정 날짜의 데이터 출력 함수
def print_aqi_for_date(target_date):
    target_date = pd.to_datetime(target_date)
    row = combined_data[combined_data['date'] == target_date]
    if row.empty:
        print(f"No data available for the date: {target_date.strftime('%Y-%m-%d')}")
    else:
        print(f"{target_date.strftime('%Y-%m-%d')} 대기질 예측 데이터")
        print(f"예상 PM25 : {round(row['Predicted_AQI_PM25'].values[0])} (Category: {row['Predicted_AQI_PM25_Category'].values[0]})")
        print(f"예상 PM10 : {round(row['Predicted_AQI_PM10'].values[0])} (Category: {row['Predicted_AQI_PM10_Category'].values[0]})")

# 특정 날짜의 AQI 데이터를 출력
target_date = '2024-09-30'
print_aqi_for_date(target_date)

In [204]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    numeric_df.rename(columns={' pm25': 'no2', ' pm10': 'so2'}, inplace=True)  # Rename columns
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

# NO2 예측 모델
X_no2 = corr_beijing_seoul[['no2_beijing', 'so2_beijing']]
y_no2 = corr_beijing_seoul['no2_seoul']

# SO2 예측 모델
X_so2 = corr_beijing_seoul[['no2_beijing', 'so2_beijing']]
y_so2 = corr_beijing_seoul['so2_seoul']

# 데이터 준비 함수
def prepare_data(X, y):
    X = X.copy()
    y = y.copy()
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.astype(float).mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.astype(float).mean(), inplace=True)
    return X.astype(float), y.astype(float)

# NO2, SO2 데이터 준비
X_no2, y_no2 = prepare_data(X_no2, y_no2)
X_so2, y_so2 = prepare_data(X_so2, y_so2)

# 모델 생성 및 학습
model_no2 = LinearRegression()
model_no2.fit(X_no2, y_no2)

model_so2 = LinearRegression()
model_so2.fit(X_so2, y_so2)

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates,
    'no2_beijing': [random.randint(50, 150) for _ in range(len(future_dates))],
    'so2_beijing': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# NO2, SO2 예측
future_X_no2 = future_beijing[['no2_beijing', 'so2_beijing']]
future_X_so2 = future_beijing[['no2_beijing', 'so2_beijing']]

future_pred_no2 = model_no2.predict(future_X_no2)
future_pred_so2 = model_so2.predict(future_X_so2)

future_beijing['Predicted_NO2'] = future_pred_no2
future_beijing['Predicted_SO2'] = future_pred_so2

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')][['date', ' pm25', ' pm10']].reset_index(drop=True)
actual_seoul.columns = ['date', 'Actual_NO2', 'Actual_SO2']

# 'Actual_NO2'와 'Actual_SO2' 열을 숫자로 변환
actual_seoul['Actual_NO2'] = pd.to_numeric(actual_seoul['Actual_NO2'], errors='coerce')
actual_seoul['Actual_SO2'] = pd.to_numeric(actual_seoul['Actual_SO2'], errors='coerce')

# 데이터 병합
combined_data = pd.merge(future_beijing, actual_seoul, on='date', how='left')

# SO2 예측 오차 범주화 함수 정의
def categorize_so2_error(error):
    if error <= 100:
        return 1
    elif error <= 200:
        return 2
    elif error <= 300:
        return 3
    elif error <= 400:
        return 4
    else:
        return 0

combined_data['NO2_Predict_Category'] = combined_data['Predicted_NO2'].apply(categorize_so2_error)
combined_data['SO2_Predict_Category'] = combined_data['Predicted_SO2'].apply(categorize_so2_error)
combined_data['NO2_Actual_Category'] = combined_data['Actual_NO2'].apply(categorize_so2_error)
combined_data['SO2_Actual_Category'] = combined_data['Actual_SO2'].apply(categorize_so2_error)

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories(row):
    if row['NO2_Predict_Category'] != row['NO2_Actual_Category']:
        row['NO2_Category_Difference'] = 1
    else:
        row['NO2_Category_Difference'] = 0
    
    if row['SO2_Predict_Category'] != row['SO2_Actual_Category']:
        row['SO2_Category_Difference'] = 1
    else:
        row['SO2_Category_Difference'] = 0
    
    return row

# COMBINED_DATA에 적용
combined_data_diff = combined_data.apply(compare_aqi_categories, axis=1)

# 2024년 한 해 동안의 데이터 필터링
total_data = combined_data_diff[(combined_data_diff['date'] >= '2024-01-01') & 
                                (combined_data_diff['date'] <= '2024-12-31')]

year_data = total_data[(total_data['date'] >= '2024-01-01') & 
                       (total_data['date'] <= '2024-06-30')]

# NO2 및 SO2의 카테고리가 같은 비율 계산
total_days = len(year_data)
same_no2_category_count = year_data[year_data['NO2_Category_Difference'] == 0].shape[0]
same_so2_category_count = year_data[year_data['SO2_Category_Difference'] == 0].shape[0]

percent_same_no2_category = (same_no2_category_count / total_days) * 100
percent_same_so2_category = (same_so2_category_count / total_days) * 100

# 결과 출력
print(f"2024년 동안 NO2의 예측 카테고리가 실제와 같은 비율: {percent_same_no2_category:.2f}%")
print(f"2024년 동안 SO2의 예측 카테고리가 실제와 같은 비율: {percent_same_so2_category:.2f}%\n")

# 특정 날짜의 AQI 데이터를 출력 함수
def print_aqi_for_date(target_date):
    target_date = pd.to_datetime(target_date)
    row = combined_data[combined_data['date'] == target_date]
    if row.empty:
        print(f"No data available for the date: {target_date.strftime('%Y-%m-%d')}")
    else:
        print(f"{target_date.strftime('%Y-%m-%d')} 대기질 예측 데이터")
        print(f"예상 NO2 : {round(row['Predicted_NO2'].values[0])} (Category: {row['NO2_Predict_Category'].values[0]})")
        print(f"예상 SO2 : {round(row['Predicted_SO2'].values[0])} (Category: {row['SO2_Predict_Category'].values[0]})")
        actual_no2 = round(row['Actual_NO2'].values[0]) if not np.isnan(row['Actual_NO2'].values[0]) else 'N/A'
        actual_so2 = round(row['Actual_SO2'].values[0]) if not np.isnan(row['Actual_SO2'].values[0]) else 'N/A'
        actual_no2_category = row['NO2_Actual_Category'].values[0] if not np.isnan(row['Actual_NO2'].values[0]) else 'N/A'
        actual_so2_category = row['SO2_Actual_Category'].values[0] if not np.isnan(row['Actual_SO2'].values[0]) else 'N/A'
        print(f"실제 NO2 : {actual_no2} (Category: {actual_no2_category})")
        print(f"실제 SO2 : {actual_so2} (Category: {actual_so2_category})")

# 특정 날짜의 AQI 데이터를 출력
target_date = '2024-12-25'
print_aqi_for_date(target_date)


2024년 동안 NO2의 예측 카테고리가 실제와 같은 비율: 79.67%
2024년 동안 SO2의 예측 카테고리가 실제와 같은 비율: 98.90%

2024-12-25 대기질 예측 데이터
예상 NO2 : 74 (Category: 1)
예상 SO2 : 40 (Category: 1)
실제 NO2 : N/A (Category: N/A)
실제 SO2 : N/A (Category: N/A)
